In [10]:
# !pip install sentence_transformers
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Linguini\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


^C


In [167]:
import nltk
import torch
import torch.nn.functional as F
import numpy as np

# from sentence_transformers import SentenceTransformer, util

from transformers import pipeline
from transformers import AutoTokenizer, AutoModel
from torch.nn.functional import cosine_similarity

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Linguini\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Linguini\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

# Try out Sentence Transformers

In [232]:
# model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
'''
Good model, but sometimes classifies it too low
'''
# model = SentenceTransformer("jxm/cde-small-v1", trust_remote_code = True)
'''
Too slow
'''
# model = SentenceTransformer("all-mpnet-base-v2", trust_remote_code = True)
'''
It misses the mark a lot
'''
# model = SentenceTransformer("paraphrase-MiniLM-L12-v2", trust_remote_code = True)


from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
# model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

classifier = pipeline(
    "zero-shot-classification",
    model="typeform/distilbert-base-uncased-mnli"
)
tokenizer = AutoTokenizer.from_pretrained("typeform/distilbert-base-uncased-mnli")


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Device set to use cuda:0
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can 

In [202]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def max_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    masked_embeddings = token_embeddings * input_mask_expanded
    return torch.max(masked_embeddings, dim=1).values

def sentence_similarity(text, topics):
    encoded_text = tokenizer(extract_keywords(text), padding=True, truncation=True, return_tensors="pt")
    encoded_topics = tokenizer(topics, padding=True, truncation=True, return_tensors="pt")

    text_output = model(**encoded_text)
    topics_output = model(**encoded_topics)

    text_embedding = mean_pooling(text_output, encoded_text["attention_mask"])
    topic_embeddings = mean_pooling(topics_output, encoded_topics["attention_mask"])

    similarities = torch.nn.functional.cosine_similarity(text_embedding, topic_embeddings)
    return similarities

def split_text(text, max_length=512):
    encoded_text = tokenizer.encode(text)
    chunks = [encoded_text[i:i + max_length] for i in range(0, len(encoded_text), max_length)]
    return [tokenizer.decode(chunk) for chunk in chunks]

def expand_context(topics):
    return [f"This text is about {word}." for word in topics]

def extract_keywords(text):
    words = word_tokenize(text)
    keywords = [word for word in words if word.lower() not in stopwords.words("english")]
    return " ".join(keywords)

def check_long_text(text, topics, threshold=0.25, k=3):
    split_list = split_text(text, max_length=100)
    results = []
    for chunk in split_list:
        # Check the similarities
        similarities = sentence_similarity(chunk, topics)
        # print(similarities)
        results.append(similarities.detach().cpu().numpy())

    scores = np.array(results)
    top_k_scores = np.sort(scores, axis=0)[-min(k, scores.shape[0]):]
    zipped = zip(np.mean(top_k_scores, axis=0), topics)
    sorted_zipped = sorted(zipped, key=lambda x: x[0], reverse=True)
    arr, labels = zip(*sorted_zipped)
    # print("Final arr:", np_arr)
    print("Labels:", labels)
    print("Scores:", arr)



    if max(arr) >= threshold:
      return True
    return False

def combine_topics(topics):
    return f"This text is about {', '.join(topics[:-1])}, and {topics[-1]}."

def check_long_text_combined(text, topics, threshold=0.25, k=3):
    split_list = split_text(text, max_length=512)
    results = []
    for chunk in split_list:
        # Check the similarities
        similarities = sentence_similarity(chunk, combine_topics(topics))
        # print(similarities)
        results.append(similarities.detach().cpu().numpy())
    scores = np.array(results)
    top_k_scores = np.sort(scores, axis=0)[-min(k, scores.shape[0]):]
    print("Sentence: ", combine_topics(topics))
    print("Top k mean:", np.mean(top_k_scores, axis=0))
    print("Scores:", top_k_scores)
    if max(top_k_scores) >= threshold:
      return True
    return False


In [227]:
import torch
def split_text_distilbert(text, max_length=512):
    tokens = tokenizer.tokenize(text)
    chunks = [tokens[i:i + max_length] for i in range(0, len(tokens), max_length)]
    return [tokenizer.convert_tokens_to_string(chunk) for chunk in chunks]


import torch
import numpy as np

def classify_with_distilbert(text, topics, k=3, threshold=0.5, batch_size=8):
    hypotheses = [f"This text is about {topic}." for topic in topics]
    
    # Split the text into chunks
    split_list = split_text_distilbert(text)
    results = []
    
    # Process chunks in batches
    for i in range(0, len(split_list), batch_size):
        batch_chunks = split_list[i:i + batch_size]
        inputs = tokenizer(
            batch_chunks * len(hypotheses),  # Repeat chunks for all hypotheses
            hypotheses * len(batch_chunks),  # Repeat hypotheses for all chunks
            return_tensors="pt",
            truncation=True,
            padding=True
        )
        
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Reshape logits: [batch_size, len(topics)] and store results
        batch_logits = outputs.logits.view(len(batch_chunks), len(hypotheses), -1)
        results.append(batch_logits.cpu().numpy())
    
    # Concatenate all batch results
    aggregated_logits = np.mean(np.concatenate(results, axis=0), axis=0)
    
    # Apply softmax to aggregated logits
    final_scores = torch.softmax(torch.tensor(aggregated_logits), dim=-1).numpy()
    
    # Map scores to topics
    topic_scores = zip(final_scores[:, 0], topics)
    sorted_topic_scores = sorted(topic_scores, key=lambda x: x[0], reverse=True)
    
    # Unzip sorted results
    scores, labels = zip(*sorted_topic_scores)
    
    print("Labels:", labels)
    print("Scores:", scores)
    
    return max(scores) >= threshold


## Try out long context models

### Google BigBird

In [ ]:
from transformers import AutoTokenizer, AutoModel
from torch.nn.functional import cosine_similarity

# tokenizer = AutoTokenizer.from_pretrained("google/bigbird-roberta-base")
# large_model = AutoModel.from_pretrained("google/bigbird-roberta-base")

tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")
large_model = AutoModel.from_pretrained("allenai/longformer-base-4096")

def get_embeddings(text, model, tokenizer, max_length=1024):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length)
    outputs = model(**inputs)

    embedding = outputs.last_hidden_state.mean(dim=1)
    return embedding

def compute_similarity(long_text, topics, model, tokenizer, threshold=0.3):
    text_embedding = get_embeddings(long_text, model, tokenizer)
    topic_embeddings = [get_embeddings(topic, model, tokenizer) for topic in topics]

    similarities = [cosine_similarity(text_embedding, topic_embedding).item() for topic_embedding in topic_embeddings]
    print(similarities)
    is_relevant = any(score >= threshold for score in similarities)

    return is_relevant, similarities

# Try out better classification models
## Facebook/BART-large-mnli


In [211]:
# classifier = pipeline("zero-shot-classification",
#                       model="microsoft/deberta-large-mnli")
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device=-1)
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")


Device set to use cpu


In [239]:
def chunk_text(text, max_length=512):
  encoded_text = tokenizer.encode(text)
  chunks = [encoded_text[i:i + max_length] for i in range(0, len(encoded_text), max_length)]
  return [tokenizer.decode(chunk) for chunk in chunks]

def check_long_text_facebook(text, topics, threshold=0.75, k=3):
    # split_list = split_text(text)
    results = classifier(chunk_text(text, max_length=100), topics, multi_label=True)
    # scores = np.array([result['scores'] for result in results])
    # aggregated_scores = np.mean(scores, axis=0)

    consistent_labels = sorted(topics)
    final_arr = []
    for result in results:
        label_score_map = dict(zip(result['labels'], result['scores']))
        aligned_scores = [label_score_map[label] for label in consistent_labels]
        final_arr.append(aligned_scores)

    np_arr = np.array(final_arr)
    
    top_k_scores = np.sort(np_arr, axis=0)[-min(k, np_arr.shape[0]):]
    zipped = zip(np.mean(top_k_scores, axis=0), consistent_labels)
    sorted_zipped = sorted(zipped, key=lambda x: x[0], reverse=True)
    arr, labels = zip(*sorted_zipped)
    # print("Final arr:", np_arr)
    print("Labels:", labels)
    print("Scores:", arr)
    if max(arr) >= threshold:
      return True
    return False

# Test data

## Test: Wikipedia Article

In [12]:
text1 = '''
The nullification crisis was a sectional political crisis in the United States in 1832 and 1833, during the presidency of Andrew Jackson, which involved a confrontation between the state of South Carolina and the federal government. It ensued after South Carolina declared the federal Tariffs of 1828 and 1832 unconstitutional and therefore null and void within the sovereign boundaries of the state. However, courts at the state and federal level, including the U.S. Supreme Court, repeatedly have rejected the theory of nullification by states.[1][2]

The controversial and highly protective Tariff of 1828 was enacted into law during the presidency of John Quincy Adams. The tariff was strongly opposed in the South, since it was perceived to put an unfair tax burden on the Southern agrarian states that imported most manufactured goods. The tariff's opponents expected that Jackson's election as president would result in its significant reduction.[3] When the Jackson administration failed to take any action to address their concerns, South Carolina's most radical faction began to advocate that the state nullify the tariff. They subscribed to the legal theory that if a state believed a federal law unconstitutional, it could declare the law null and void in the state. In Washington, an open split on the issue occurred between Jackson and Vice President John C. Calhoun, a native South Carolinian and the most effective proponent of the constitutional theory of state nullification.[4]

On July 1, 1832, before Calhoun resigned the vice presidency to run for the Senate, where he could more effectively defend nullification,[5] Jackson signed into law the Tariff of 1832. This compromise tariff received the support of most Northerners and half the Southerners in Congress.[6] South Carolina remained unsatisfied, and on November 24, 1832, a state convention adopted the Ordinance of Nullification, which declared that the Tariffs of 1828 and 1832 were unconstitutional and unenforceable in South Carolina after February 1, 1833.[7] South Carolina initiated military preparations to resist anticipated federal enforcement,[8] but on March 1, 1833, Congress passed both the Force Bill—authorizing the president to use military forces against South Carolina—and a new negotiated tariff, the Compromise Tariff of 1833, which was satisfactory to South Carolina. The South Carolina convention reconvened and repealed its Nullification Ordinance on March 15, 1833, but three days later, nullified the Force Bill as a symbolic gesture of principle.

The crisis was over, and both sides found reasons to claim victory. The tariff rates were reduced and stayed low to the satisfaction of the South, but the states' rights doctrine of nullification remained controversial. By the 1850s, the issues of the expansion of slavery into the western territories and the threat of the Slave Power became the central issues in the nation.[9]
Background (1787–1816)

The historian Richard E. Ellis wrote:

    By creating a national government with the authority to act directly upon individuals, by denying to the state many of the prerogatives that they formerly had, and by leaving open to the central government the possibility of claiming for itself many powers not explicitly assigned to it, the Constitution and Bill of Rights as finally ratified substantially increased the strength of the central government at the expense of the states.[10]

The extent of this change and the problem of the actual distribution of powers between state and the federal governments would be a matter of political and ideological discussion through the Civil War as well as afterwards.[11] In the early 1790s the debate centered on Alexander Hamilton's nationalistic financial program versus Jefferson's democratic and agrarian program, a conflict that led to the formation of two opposing national political parties. Later in the decade the Alien and Sedition Acts led to the states' rights position being articulated in the Kentucky and Virginia Resolutions.[12] The Kentucky Resolutions, written by Thomas Jefferson, contained the following, which has often been cited as a justification for both nullification and secession:

    ... that in cases of an abuse of the delegated powers, the members of the general government, being chosen by the people, a change by the people would be the constitutional remedy; but, where powers are assumed which have not been delegated, a nullification of the act is the rightful remedy: that every State has a natural right in cases not within the compact, (casus non fœderis) to nullify of their own authority all assumptions of power by others within their limits: that without this right, they would be under the dominion, absolute and unlimited, of whosoever might exercise this right of judgment for them: that nevertheless, this commonwealth, from motives of regard and respect for its co-States, has wished to communicate with them on the subject: that with them alone it is proper to communicate, they alone being parties to the compact, and solely authorized to judge in the last resort of the powers exercised under it ...[13]

The Virginia Resolutions, written by James Madison, hold a similar argument:

    The resolutions, having taken this view of the Federal compact, proceed to infer that, in cases of a deliberate, palpable, and dangerous exercise of other powers, not granted by the said compact, the States, who are parties thereto, have the right, and are in duty bound to interpose to arrest the evil, and for maintaining, within their respective limits, the authorities, rights, and liberties appertaining to them. ... The Constitution of the United States was formed by the sanction of the States, given by each in its sovereign capacity. It adds to the stability and dignity, as well as to the authority of the Constitution, that it rests on this solid foundation. The States, then, being parties to the constitutional compact, and in their sovereign capacity, it follows of necessity that there can be no tribunal above their authority to decide, in the last resort, whether the compact made by them be violated; and, consequently, as parties to it, they must themselves decide, in the last resort, such questions as may be of sufficient magnitude to require their interposition.[14]

Historians differ over the extent to which either resolution advocated the doctrine of nullification. Historian Lance Banning wrote, "The legislators of Kentucky (or more likely, John Breckinridge, the Kentucky legislator who sponsored the resolution) deleted Jefferson's suggestion that the rightful remedy for federal usurpation was a "nullification" of such acts by each state acting on its own to prevent their operation within its respective borders. Rather than suggesting individual, although concerted, measures of this sort, Kentucky was content to ask its sisters to unite in declarations that the acts were "void and of no force", and in "requesting their appeal" at the succeeding session of the Congress."[15] The key sentence, and the word "nullification" was used in supplementary Resolutions passed by Kentucky in 1799.[16]

Madison's judgment is clearer. He was chairman of a committee of the Virginia Legislature, which issued a book-length Report on the Resolutions of 1798, published in 1800 after they had been decried by several states. This asserted that the state did not claim legal force. "The declarations in such cases are expressions of opinion, unaccompanied by other effect than what they may produce upon opinion, by exciting reflection. The opinions of the judiciary, on the other hand, are carried into immediate effect by force." If the states collectively agreed in their declarations, there were several methods by which it might prevail, from persuading Congress to repeal the unconstitutional law, to calling a constitutional convention, as two-thirds of the states may.[17] When, at the time of the nullification crisis, he was presented with the Kentucky resolutions of 1799, he argued that the resolutions themselves were not Jefferson's words, and that Jefferson meant this not as a constitutional, but as a revolutionary right.[18]

Madison biographer Ralph Ketcham wrote:

    Though Madison agreed entirely with the specific condemnation of the Alien and Sedition Acts, with the concept of the limited delegated power of the general government, and even with the proposition that laws contrary to the Constitution were illegal, he drew back from the declaration that each state legislature had the power to act within its borders against the authority of the general government to oppose laws the legislature deemed unconstitutional.[19]

Historian Sean Wilentz explains the widespread opposition to these resolutions:

    Several states followed Maryland's House of Delegates in rejecting the idea that any state could, by legislative action, even claim that a federal law was unconstitutional, and suggested that any effort to do so was treasonous. A few northern states, including Massachusetts, denied the powers claimed by Kentucky and Virginia and insisted that the Sedition law was perfectly constitutional . ... Ten state legislatures with heavy Federalist majorities from around the country censured Kentucky and Virginia for usurping powers that supposedly belonged to the federal judiciary. Northern Republicans supported the resolutions' objections to the alien and sedition acts, but opposed the idea of state review of federal laws. Southern Republicans outside Virginia and Kentucky were eloquently silent about the matter, and no southern legislature heeded the call to battle.[20]

Portrait of Thomas Jefferson by Rembrandt Peale, 1800

The election of 1800 was a turning point in national politics, as the Federalists were replaced by the Democratic-Republican Party led by Jefferson, but the four presidential terms spanning the period from 1800 to 1817 "did little to advance the cause of states' rights and much to weaken it." Over Jefferson's opposition, the power of the federal judiciary, led by Federalist Chief Justice John Marshall, increased. Jefferson expanded federal powers with the acquisition of the Louisiana Territory and his use of a national embargo designed to prevent involvement in a European war. Madison in 1809 used national troops to enforce a Supreme Court decision in Pennsylvania, appointed an "extreme nationalist" in Joseph Story to the Supreme Court, signed the bill creating the Second Bank of the United States, and called for a constitutional amendment to promote internal improvements.[21]

Opposition to the War of 1812 was centered in New England. Delegates to a convention in Hartford, Connecticut, met in December 1814 to consider a New England response to Madison's war policy. The debate allowed many radicals to argue the cause of states' rights and state sovereignty. In the end, moderate voices dominated and the final product was not secession or nullification, but a series of proposed constitutional amendments.[22] Identifying the South's domination of the government as the cause of much of their problems, the proposed amendments included "the repeal of the three-fifths clause, a requirement that two-thirds of both houses of Congress agree before any new state could be admitted to the Union, limits on the length of embargoes, and the outlawing of the election of a president from the same state to successive terms, clearly aimed at the Virginians."[23] The war was over before the proposals were submitted to President Madison.
James Madison

After the conclusion of the War of 1812 Sean Wilentz notes:

    Madison's speech [his 1815 annual message to Congress] affirmed that the war had reinforced the evolution of mainstream Republicanism, moving it further away from its original and localist assumptions. The war's immense strain on the treasury led to new calls from nationalist Republicans for a national bank. The difficulties in moving and supplying troops exposed the wretchedness of the country's transportation links, and the need for extensive new roads and canals. A boom in American manufacturing during the prolonged cessation of trade with Britain created an entirely new class of enterprisers, most of them tied politically to the Republicans, who might not survive without tariff protection. More broadly, the war reinforced feelings of national identity and connection.[24]

This spirit of nationalism was linked to the tremendous growth and economic prosperity of this postwar era. However in 1819, the nation suffered its first financial panic and the 1820s turned out to be a decade of political turmoil that again led to fierce debates over competing views of the exact nature of American federalism. The "extreme democratic and agrarian rhetoric" that had been so effective in 1798 led to renewed attacks on the "numerous market-oriented enterprises, particularly banks, corporations, creditors, and absentee landholders".[25]
Tariffs (1816–1828)

The Tariff of 1816 had some protective features, and it received support throughout the nation, including that of John C. Calhoun and fellow South Carolinian William Lowndes.[26] The first explicitly protective tariff linked to a specific program of internal improvements was the Tariff of 1824.[27] Sponsored by Henry Clay, this tariff provided a general level of protection at 35% ad valorem (compared to 25% with the 1816 act) and hiked duties on iron, woolens, cotton, hemp, and wool and cotton bagging. The bill barely passed the federal House of Representatives by a vote of 107 to 102. The Middle states and Northwest supported the bill, the South and Southwest opposed it, and New England split its vote with a majority opposing it. In the Senate, the bill, with the support of Tennessee Senator Andrew Jackson, passed by four votes, and President James Monroe, the Virginia heir to the Jefferson-Madison control of the White House, signed the bill on March 25, 1824.[28] Daniel Webster of Massachusetts led the New England opposition to this tariff.[29]

Protest against the prospect and the constitutionality of higher tariffs began in 1826 and 1827 with William Branch Giles, who had the Virginia legislature pass resolutions denying the power of Congress to pass protective tariffs, citing the Virginia Resolutions of 1798 and James Madison's 1800 defense of them. Madison denied both the appeal to nullification and the unconstitutionality; he had always held that the power to regulate commerce included protection. Jefferson had, at the end of his life, written against protective tariffs.[30]
Portrait of Martin Van Buren

The Tariff of 1828 was largely the work of Martin Van Buren (although Silas Wright Jr. of New York prepared the main provisions) and was partly a political ploy to elect Andrew Jackson President. Van Buren calculated that the South would vote for Jackson regardless of the issues, so he ignored their interests in drafting the bill. New England, he thought, was just as likely to support the incumbent John Quincy Adams, so the bill levied heavy taxes on raw materials consumed by New England such as hemp, flax, molasses, iron, and sail duck. With an additional tariff on iron to satisfy Pennsylvania interests, Van Buren expected the tariff to help deliver Pennsylvania, New York, Missouri, Ohio, and Kentucky to Jackson. Over opposition from the South and some from New England, the tariff was passed with the full support of many Jackson supporters in Congress and signed by President Adams in early 1828.[31]

As expected, Jackson and his running mate John Calhoun carried the entire South with overwhelming numbers in every state but Louisiana, where Adams drew 47% of the vote in a losing effort. But many Southerners became dissatisfied as Jackson, in his first two annual messages to Congress, failed to launch a strong attack on the tariff. Historian William J. Cooper Jr. writes:

    The most doctrinaire ideologues of the Old Republican group [supporters of the Jefferson and Madison position in the late 1790s] first found Jackson wanting. These purists identified the tariff of 1828, the hated Tariff of Abominations, as the most heinous manifestation of the nationalist policy they abhorred. That protective tariff violated their constitutional theory, for, as they interpreted the document, it gave no permission for a protective tariff. Moreover, they saw protection as benefiting the North and hurting the South.[32]

South Carolina background (1819–1828)
John C. Calhoun

South Carolina had been adversely affected by the national economic decline of the 1820s. During this decade, the population decreased by 56,000 whites and 30,000 slaves, out of a total free and slave population of 580,000. The whites left for better places; they took slaves with them or sold them to traders moving slaves to the Deep South for sale.[33]

Historian Richard E. Ellis describes the situation:

    Throughout the colonial and early national periods, South Carolina had sustained substantial economic growth and prosperity. This had created an extremely wealthy and extravagant low country aristocracy whose fortunes were based first on the cultivation of rice and indigo, and then on cotton. Then the state was devastated by the Panic of 1819. The depression that followed was more severe than in almost any other state of the Union. Moreover, competition from the newer cotton producing areas along the Gulf Coast, blessed with fertile lands that produced a higher crop-yield per acre, made recovery painfully slow. To make matters worse, in large areas of South Carolina slaves vastly outnumbered whites, and there existed both considerable fear of slave rebellion and a growing sensitivity to even the smallest criticism of "the peculiar institution."[34]

George McDuffie

State leaders, led by states' rights advocates such as William Smith and Thomas Cooper, blamed most of the state's economic problems on the Tariff of 1816 and national internal improvement projects. Soil erosion and competition from the New Southwest were also very significant reasons for the state's declining fortunes.[35] George McDuffie was a particularly effective speaker for the anti-tariff forces, and he popularized the Forty Bale theory. McDuffie argued that the 40% tariff on cotton finished goods meant that "the manufacturer actually invades your barns, and plunders you of 40 out of every 100 bales that you produce." Mathematically incorrect, this argument still struck a nerve with his constituency. Nationalists such as Calhoun were forced by the increasing power of such leaders to retreat from their previous positions and adopt, in the words of Ellis, "an even more extreme version of the states' rights doctrine" in order to maintain political significance within South Carolina.[36]
Thomas Bee's House, Charleston, circa 1730: The Nullification Movement that split the nation started here in 1832. John C. Calhoun, S.C. Governor Robert Hayne, General James Hamilton and other leaders drafted the Nullification Papers in the second-floor drawing room.

South Carolina's first effort at nullification occurred in 1822. Its planter class believed that free black sailors had assisted Denmark Vesey in his planned slave rebellion. South Carolina passed the Negro Seamen Act, which required all black foreign seamen to be imprisoned while their ships were docked in Charleston. The United Kingdom strongly objected, especially as it was recruiting more Africans as sailors. Worse, if the captains did not pay the fees to cover the cost of jailing, South Carolina would sell the sailors into slavery. Other Southern states also passed laws against free black sailors.[37]

Supreme Court Justice William Johnson, in his capacity as a circuit judge, declared the South Carolina law as unconstitutional since it violated the United States' treaties with the United Kingdom. The South Carolina Senate announced that the judge's ruling was invalid and that the act would be enforced. The federal government did not attempt to carry out Johnson's decision.[38]
Route to nullification in South Carolina (1828–1832)
Joel Roberts Poinsett, South Carolina Unionist leader

Historian Avery Craven argues that, for the most part, the debate from 1828 to 1832 was a local South Carolina affair. The state's leaders were not united and the sides were roughly equal. The western part of the state and a faction in Charleston, led by Joel Poinsett and Thomas Smith Grimké, remained loyal to the Union. Only in small part was the conflict between "a National North against a States'-right South".[39]

After the final vote on the Tariff of 1828, South Carolina's congressional delegation held two caucuses, the second at the home of Senator Robert Y. Hayne. They were rebuffed in their efforts to coordinate a united Southern response and focused on how their state representatives would react. While many agreed with McDuffie that tariff policy could lead to secession, they all agreed that, as much as possible, the issue should be kept out of the upcoming presidential election. Calhoun, while not at this meeting, served as a moderating influence. He felt that the first step in reducing the tariff was to defeat Adams and his supporters in the upcoming election. William C. Preston, on behalf of the South Carolina legislature, asked Calhoun to prepare a report on the tariff situation. Calhoun readily accepted and in a few weeks had a 35,000-word draft of what would become his "Exposition and Protest".[40]

Calhoun's "Exposition" was completed late in 1828. He argued that the tariff of 1828 was unconstitutional because it favored manufacturing over commerce and agriculture. He believed the tariff power could be used only to generate revenue, not to provide protection from foreign competition for American industries, and that the people of a state or several states, acting in a democratically elected convention, had the power to veto any act of the federal government that violated the Constitution. This veto, the core of the doctrine of nullification, was explained by Calhoun in the Exposition:

    If it be conceded, as it must be by every one who is the least conversant with our institutions, that the sovereign powers delegated are divided between the General and State Governments, and that the latter hold their portion by the same tenure as the former, it would seem impossible to deny to the States the right of deciding on the infractions of their powers, and the proper remedy to be applied for their correction. The right of judging, in such cases, is an essential attribute of sovereignty, of which the States cannot be divested without losing their sovereignty itself, and being reduced to a subordinate corporate condition. In fact, to divide power, and to give to one of the parties the exclusive right of judging of the portion allotted to each, is, in reality, not to divide it at all; and to reserve such exclusive right to the General Government (it matters not by what department to be exercised), is to convert it, in fact, into a great consolidated government, with unlimited powers, and to divest the States, in reality, of all their rights, It is impossible to understand the force of terms, and to deny so plain a conclusion.[41]

The report also detailed the specific southern grievances over the tariff that led to the current dissatisfaction.[42] Fearful that "hotheads" such as McDuffie might force the legislature into taking drastic action against the federal government, historian John Niven describes Calhoun's political purpose in the document:

    All through that hot and humid summer, emotions among the vociferous planter population had been worked up to a near-frenzy of excitement. The whole tenor of the argument built up in the "Exposition" was aimed to present the case in a cool, considered manner that would dampen any drastic moves yet would set in motion the machinery for repeal of the tariff act. It would also warn other sections of the Union against any future legislation that an increasingly self-conscious South might consider punitive, especially on the subject of slavery.[43]

The report was submitted to the state legislature, which had 5,000 copies printed and distributed. Calhoun, who still had designs on succeeding Jackson as president, was not identified as the author, but word on this soon leaked out. The legislature took no action on the report at that time.[44]

In the summer of 1828, Robert Barnwell Rhett, soon to be considered the most radical of the South Carolinians, entered the fray over the tariff. As a state representative, Rhett called for the governor to convene a special session of the legislature. An outstanding orator, Rhett appealed to his constituents to resist the majority in Congress. He addressed the danger of doing nothing:

    But if you are doubtful of yourselves—if you are not prepared to follow up your principles wherever they may lead, to their very last consequence—if you love life better than honor,—prefer ease to perilous liberty and glory; awake not! Stir not!—Impotent resistance will add vengeance to your ruin. Live in smiling peace with your insatiable Oppressors, and die with the noble consolation that your submissive patience will survive triumphant your beggary and despair.[45]

Rhett's rhetoric about revolution and war was too radical in the summer of 1828 but, with the election of Jackson assured, James Hamilton Jr. on October 28 in the Colleton County Courthouse in Walterborough "launched the formal nullification campaign."[46] Renouncing his former nationalism, Hamilton warned the people that "Your task-master must soon become a tyrant, from the very abuses and corruption of the system, without the bowels of compassion, or a jot of human sympathy." He called for implementation of Jefferson's "rightful remedy" of nullification. Hamilton sent a copy of the speech directly to President-elect Jackson. But despite a statewide campaign by Hamilton and McDuffie, a proposal to call a nullification convention in 1829 was defeated by the South Carolina legislature meeting at the end of 1828. State leaders such as Calhoun, Hayne, Smith, and William Drayton all remained publicly noncommittal or opposed to nullification for the next couple of years.[47]

The division in the state between radicals and conservatives continued through 1829 and 1830. After the failure of a state project to arrange financing of a railroad within the state to promote internal trade, the state petitioned Congress to invest $250,000 in the company trying to build it. After Congress tabled the measure, debate in South Carolina resumed between those who wanted state investment and those who wanted to work to get Congress's support. The debate demonstrated that a significant minority of the state did have an interest in Clay's American System. The effect of the Webster–Hayne debate was to energize the radicals, and some moderates started to move in their direction.[48]

The state election campaign of 1830 focused on the tariff issue and the need for a state convention. On the defensive, radicals underplayed the intent of the convention as pro-nullification. When voters were presented with races where an unpledged convention was the issue, the radicals generally won. When conservatives effectively characterized the race as being about nullification, the radicals lost. The October election was narrowly carried by the radicals, although the blurring of the issues left them without any specific mandate.[49] In South Carolina, the governor was selected by the legislature, which chose James Hamilton, the leader of the radical movement, and fellow radical Henry L. Pinckney as speaker of the South Carolina House. For the open Senate seat, the legislature chose the more radical Stephen Decatur Miller over William Smith.[50]

With radicals in leading positions, in 1831 they began to capture momentum. State politics became sharply divided along Nullifier and Unionist lines. Still, the margin in the legislature fell short of the two-thirds majority needed for a convention. Many of the radicals felt that convincing Calhoun of the futility of his plans for the presidency would lead him into their ranks. Calhoun, meanwhile, had concluded that Van Buren was establishing himself as Jackson's heir apparent. At Hamilton's prompting, McDuffie made a three-hour speech in Charleston demanding nullification of the tariff at any cost. In the state, the success of McDuffie's speech seemed to open up the possibilities of both military confrontation with the federal government and civil war within the state. With silence no longer an acceptable alternative, Calhoun looked for the opportunity to take control of the antitariff faction in the state; by June he was preparing what would be known as his Fort Hill Address.[51]

Published on July 26, 1831, the address repeated and expanded the positions Calhoun had made in the "Exposition". While the logic of much of the speech was consistent with the states' rights position of most Jacksonians, and even Daniel Webster remarked that it "was the ablest and most plausible, and therefore the most dangerous vindication of that particular form of Revolution", the speech still placed Calhoun clearly in a nullified camp. Within South Carolina, his gestures at moderation in the speech were drowned out as planters received word of the Nat Turner insurrection in Virginia. Calhoun was not alone in finding a connection between the abolition movement and the sectional aspects of the tariff issue.[52] It confirmed for Calhoun what he had written in a September 11, 1830, letter:

    I consider the tariff act as the occasion, rather than the real cause of the present unhappy state of things. The truth can no longer be disguised, that the peculiar institution of the Southern States and the consequent direction which that and her soil have given to her industry, has placed them in regard to taxation and appropriations in opposite relation to the majority of the Union, against the danger of which, if there be no protective power in the reserved rights of the states they must in the end be forced to rebel, or, submit to have their paramount interests sacrificed, their domestic institutions subordinated by Colonization and other schemes, and themselves and children reduced to wretchedness.[53]

From this point, the nullifiers accelerated their organization and rhetoric. In July 1831, the States Rights and Free Trade Association was formed in Charleston and expanded throughout the state. Unlike state political organizations in the past, which were led by the South Carolina planter aristocracy, this group appealed to all segments of the population, including non-slaveholder farmers, small slaveholders, and the Charleston non-agricultural class. Governor Hamilton was instrumental in seeing that the association, which was both a political and a social organization, expanded throughout the state. In the winter of 1831 and spring of 1832, Hamilton held conventions and rallies throughout the state to mobilize the nullification movement. The conservatives were unable to match the radicals in organization or leadership.[54]

The state elections of 1832 were "charged with tension and bespattered with violence," and "polite debates often degenerated into frontier brawls." Unlike the previous year's election, the choice was clear between nullifiers and unionists. The nullifiers won and on October 20, 1832, Hamilton called the legislature into a special session to consider a convention. The legislative vote was 96–25 in the House and 31–13 in the Senate.[55]

In November 1832, the Nullification Convention met. The convention declared the tariffs of 1828 and 1832 unconstitutional and unenforceable within the state of South Carolina after February 1, 1833. It was asserted that attempts to use force to collect the taxes would lead to the state's secession. Robert Hayne, who succeeded Hamilton as governor in 1833, established a 2,000-man group of mounted minutemen and 25,000 infantry who would march to Charleston in the event of a military conflict. These troops were to be armed with $100,000 in arms purchased in the North.[56]

The enabling legislation passed by the legislature was carefully constructed to avoid clashes if at all possible and create an aura of legality in the process. To avoid conflicts with Unionists, it allowed importers to pay the tariff if they desired. Other merchants could pay the tariff by obtaining a paper tariff bond from the customs officer. They would then refuse to pay the bond when due, and if the customs official seized the goods, the merchant would file for a writ of replevin to recover the goods in state court. Customs officials who refused to return the goods (by placing them under the protection of federal troops) would be civilly liable for twice the value of the goods. To ensure that state officials and judges supported the law, a "test oath" would be required for all new state officials, binding them to support the ordinance of nullification.[57]

Governor Hayne in his inaugural address announced South Carolina's position:

    If the sacred soil of Carolina should be polluted by the footsteps of an invader, or be stained with the blood of her citizens, shed in defense, I trust in Almighty God that no son of hers ... who has been nourished at her bosom ... will be found raising a parricidal arm against our common mother. And even should she stand ALONE in this great struggle for constitutional liberty ... that there will not be found, in the wider limits of the state, one recreant son who will not fly to the rescue, and be ready to lay down his life in her defense.[58]

Washington, D.C. (1828–1832)
Official White House portrait of Andrew Jackson

When President Jackson took office in March 1829, he was well aware of the turmoil created by the "Tariff of Abominations". While he may have abandoned some of his earlier beliefs that had allowed him to vote for the Tariff of 1824, he still felt protectionism was justified for products essential to military preparedness and did not believe that the current tariff should be reduced until the national debt was fully paid off. He addressed the issue in his inaugural address and his first three messages to Congress, but offered no specific relief. In December 1831, with the proponents of nullification in South Carolina gaining momentum, Jackson recommended "the exercise of that spirit of concession and conciliation which has distinguished the friends of our Union in all great emergencies."[59] But on the constitutional issue of nullification, despite his strong beliefs in states' rights, Jackson did not waver.

Calhoun's "Exposition and Protest" started a national debate on the doctrine of nullification. The leading proponents[60] of the nationalistic view included Daniel Webster, Supreme Court Justice Joseph Story, Judge William Alexander Duer, John Quincy Adams, Nathaniel Chipman, and Nathan Dane. They rejected the compact theory advanced by Calhoun, claiming that the Constitution was the product of the people, not the states. According to the nationalist position, the Supreme Court had the final say on legislation's constitutionality, and the national union was perpetual and had supreme authority over individual states.[61] The nullifiers, on the other hand, asserted that the central government was not the ultimate arbiter of its own power, and that the states, as the contracting entities, could judge for themselves what was constitutional. While Calhoun's "Exposition" claimed that nullification was based on the reasoning behind the Kentucky and Virginia Resolutions, an aging James Madison in an August 28, 1830, letter to Edward Everett, intended for publication, disagreed. Madison wrote, denying that any individual state could alter the compact:[62]

    Can more be necessary to demonstrate the inadmissibility of such a doctrine than that it puts it in the power of the smallest fraction over 1/4 of the U. S.—that is, of 7 States out of 24—to give the law and even the Constn. to 17 States, each of the 17 having as parties to the Constn. an equal right with each of the 7 to expound it & to insist on the exposition. That the 7 might, in particular instances be right and the 17 wrong, is more than possible. But to establish a positive & permanent rule giving such a power to such a minority over such a majority, would overturn the first principle of free Govt. and in practice necessarily overturn the Govt. itself.[63]

Webster Replying to Hayne by George P.A. Healy

Part of the South's strategy to force repeal of the tariff was to arrange an alliance with the West. Under the plan, the South would support the West's demand for free lands in the public domain if the West supported repeal of the tariff. With this purpose, Robert Hayne took the floor on the Senate in early 1830, beginning "the most celebrated debate in the Senate's history." Daniel Webster's response shifted the debate, subsequently styled the Webster-Hayne debates, from the specific issue of western lands to a general debate on the very nature of the United States. Webster's position differed from Madison's: Webster asserted that the people of the United States acted as one aggregate body, while Madison held that the people of the several states acted collectively. John Rowan spoke against Webster on that issue, and Madison wrote, congratulating Webster, but explaining his own position.[64] The debate presented the fullest articulation of the differences over nullification, and 40,000 copies of Webster's response, which concluded with "liberty and Union, now and forever, one and inseparable", were distributed nationwide.[65]

Many people expected Jackson to side with Hayne, but once the debate shifted to secession and nullification, he sided with Webster. On April 13, 1830, at the traditional Democratic Party celebration honoring Jefferson's birthday, Jackson chose to make his position clear. In a battle of toasts, Hayne proposed, "The Union of the States, and the Sovereignty of the States." Jackson's response, when his turn came, was, "Our Federal Union: It must be preserved." To those attending, the effect was dramatic. Calhoun responded with his own toast, in a play on Webster's closing remarks in the earlier debate, "The Union. Next to our liberty, the most dear." Finally, Van Buren offered, "Mutual forbearance and reciprocal concession. Through their agency the Union was established. The patriotic spirit from which they emanated will forever sustain it."

Van Buren wrote in his autobiography of Jackson's toast, "The veil was rent—the incantations of the night were exposed to the light of day." Senator Thomas Hart Benton, in his memoirs, wrote that the toast "electrified the country."[66] Jackson had the final word a few days later, when a visitor from South Carolina asked if Jackson had any message he wanted relayed to his friends back in the state. Jackson's reply was:

    Yes I have; please give my compliments to my friends in your State and say to them, that if a single drop of blood shall be shed there in opposition to the laws of the United States, I will hang the first man I can lay my hand on engaged in such treasonable conduct, upon the first tree I can reach.[67]

Other issues than the tariff were still being decided. In May 1830, Jackson vetoed the Maysville Road Bill, an important internal-improvements program (especially to Kentucky and Henry Clay), and then followed this with additional vetoes of other such projects shortly before Congress adjourned at the end of May. Clay used these vetoes to launch his presidential campaign.[68] In 1831, the rechartering of the Bank of the United States, with Clay and Jackson on opposite sides, reopened a long-simmering problem. This issue was featured at the December 1831 National Republican convention in Baltimore, which nominated Clay for president, and the proposal to recharter was formally introduced into Congress on January 6, 1832.[69] The Calhoun-Jackson split entered the center stage when Calhoun, as vice president presiding over the Senate, cast the tie-breaking vote to deny Van Buren the post of minister to England. Van Buren was subsequently selected as Jackson's running mate at the 1832 Democratic National Convention held in May.[70]
Portrait of Henry Clay

In February 1832, Clay, back in the Senate after a two-decade absence, made a three-day speech calling for a new tariff schedule and an expansion of his American System. In an effort to reach out to Calhoun and other Southerners, Clay's proposal provided for a $10 million revenue reduction based on the budget surplus he anticipated for the coming year. Significant protection was still part of the plan, as the reduction primarily came on imports not in competition with domestic producers. Jackson proposed an alternative that reduced overall tariffs to 28%. John Quincy Adams, now in the House of Representatives, used his Committee of Manufacturers to produce a compromise bill that, in its final form, reduced revenues by $5 million, lowered duties on noncompetitive products, and retained high tariffs on woolens, iron, and cotton products. During the political maneuvering, McDuffie's Ways and Means Committee, the normal originator of such bills, prepared a bill with drastic reduction across the board, but it went nowhere. Jackson signed the Tariff of 1832 on July 14, 1832, a few days after vetoing the Bank of the United States recharter bill. Congress adjourned after failing to override Jackson's veto.[71]

With Congress adjourned, Jackson anxiously watched events in South Carolina. The nullifiers found no significant compromise in the Tariff of 1832 and acted accordingly. Jackson heard rumors of efforts to subvert members of the army and navy in Charleston and ordered the secretaries of the army and navy to begin rotating troops and officers based on their loyalty. He ordered General Winfield Scott to prepare for military operations and ordered a naval squadron in Norfolk to prepare to go to Charleston. Jackson kept lines of communication open with unionists such as Joel Poinsett, William Drayton, and James L. Petigru and sent George Breathitt, brother of the Kentucky governor, to independently obtain political and military intelligence. After their defeat at the polls in October, Petigru advised Jackson to "Be prepared to hear very shortly of a State Convention and an act of Nullification."

On October 29, 1832, Jackson wrote to his Secretary of War, Lewis Cass:

    The attempt will be made to surprise the Forts & garrisons by the militia, and must be guarded against with vestal vigilance and any attempt by force repelled with prompt and exemplary punishment.

By mid-November, Jackson's reelection was assured.[72] On December 3, 1832, Jackson sent his fourth annual message to Congress. The message "was stridently states' rights and agrarian in its tone and thrust" and disavowed protection as anything other than a temporary expedient.[73] His intent regarding nullification, as communicated to Van Buren, was "to pass it barely in review, as a mere buble [sic], view the existing laws as competent to check and put it down." He hoped to create a "moral force" that would transcend political parties and sections. The paragraph in the message that addressed nullification was:

    It is my painful duty to state that in one quarter of the United States opposition to the revenue laws has arisen to a height which threatens to thwart their execution, if not to endanger the integrity of the Union. What ever obstructions may be thrown in the way of the judicial authorities of the General Government, it is hoped they will be able peaceably to overcome them by the prudence of their own officers and the patriotism of the people. But should this reasonable reliance on the moderation and good sense of all portions of our fellow citizens be disappointed, it is believed that the laws themselves are fully adequate to the suppression of such attempts as may be immediately made. Should the exigency arise rendering the execution of the existing laws impracticable from any cause what ever, prompt notice of it will be given to Congress, with a suggestion of such views and measures as may be deemed necessary to meet it.[74]

On December 10, Jackson issued the Proclamation to the People of South Carolina, in which he characterized the positions of the nullifiers as "impractical absurdity" and "a metaphysical subtlety, in pursuit of an impractical theory." He provided this concise statement of his belief:

    I consider, then, the power to annul a law of the United States, assumed by one State, incompatible with the existence of the Union, contradicted expressly by the letter of the Constitution, unauthorized by its spirit, inconsistent with every principle on which It was founded, and destructive of the great object for which it was formed.[75]

The language Jackson used, combined with the reports out of South Carolina, raised the spectre of military confrontation for many on both sides of the issue. A group of Democrats, led by Van Buren and Thomas Hart Benton, among others, saw the only solution to the crisis in a substantial reduction of the tariff.
Negotiation and confrontation (1833)

In apparent contradiction of his previous claim that the tariff could be enforced with existing laws, on January 16 Jackson sent his Force Bill Message to Congress. Custom houses in Beaufort and Georgetown would be closed and replaced by ships at each port. In Charleston, the custom house would be moved to either Castle Pinckney or Fort Moultrie in Charleston Harbor. Direct payment rather than bonds would be required, and federal jails would be established for violators the state refused to arrest and all cases arising under the state's nullification act could be removed to the United States Circuit Court. In the most controversial part, the militia acts of 1795 and 1807 would be revised to permit the enforcement of the customs laws by both the militia and the regular United States military. Attempts were made in South Carolina to shift the debate away from nullification by focusing instead on the proposed enforcement.[76]

The Force bill went to the Senate Judiciary Committee, chaired by Pennsylvania protectionist William Wilkins and supported by members Daniel Webster and Theodore Frelinghuysen of New Jersey; it gave Jackson everything he asked. On January 28, the Senate defeated a motion by a vote of 30 to 15 to postpone debate on the bill. All but two of the votes to delay were from the lower South and only three from this section voted against the motion. This did not signal any increased support for nullification, but did signify doubts about enforcement. To draw more votes, proposals were made to limit the duration of the coercive powers and restrict the use of force to suppressing, rather than preventing, civil disorder. In the House, the Judiciary Committee voted 4–3 to reject Jackson's request to use force. By the time Calhoun made a major speech on February 15 strongly opposing it, the Force Bill was temporarily stalled.[77]

On the tariff issue, the drafting of a compromise tariff was assigned in December to the House Ways and Means Committee, now headed by Gulian C. Verplanck. Debate on the committee's product on the House floor began in January 1833. The Verplanck tariff proposed reductions back to 1816 levels over the next two years while maintaining the basic principle of protectionism. The anti-Jackson protectionists saw this as an economic disaster that did not even allow the Tariff of 1832 to be tested and "an undignified truckling to the menaces and blustering of South Carolina." Northern Democrats did not oppose it in principle, but still demanded protection for the varying interests of their own constituents. Those sympathetic to the nullifiers wanted a specific abandonment of the principle of protectionism and were willing to offer a longer transition period as a bargaining point. The Verplanck tariff was clearly not going to be implemented.[78]

In South Carolina, efforts were being made to avoid an unnecessary confrontation. Governor Hayne ordered the 25,000 troops he had created to train at home rather than gather in Charleston. At a mass meeting in Charleston on January 21, they decided to postpone the February 1 deadline for implementing nullification, while Congress worked on a compromise tariff. At the same time, a commissioner from Virginia, Benjamin W. Leigh, arrived in Charleston bearing resolutions that criticized both Jackson and the nullifiers and offering his state as a mediator.[79]

Clay had not taken his defeat in the presidential election well and was unsure what position he could take in the tariff negotiations. His long-term concern was that Jackson was determined to kill protectionism along with the American Plan. In February, after consulting with manufacturers and sugar interests in Louisiana, who favored protection for the sugar industry, Clay started to work on a specific compromise plan. As a starting point, he accepted the nullifiers' offer of a transition period, but extended it from seven and a half years to nine years with a final target of a 20% ad valorem rate. After first securing the support of his protectionist base, Clay, through an intermediary, broached the subject with Calhoun. Calhoun was receptive, and after a private meeting with Clay at Clay's boardinghouse, negotiations proceeded.[80]

Clay introduced the negotiated tariff bill on February 12, and it was immediately referred to a select committee consisting of Clay as chairman, Felix Grundy of Tennessee, George M. Dallas of Pennsylvania, William Cabell Rives of Virginia, Webster, John M. Clayton of Delaware, and Calhoun. On February 21, the committee reported a bill to the floor of the Senate that was largely Clay's original bill. The Tariff of 1832 would continue except that reduction of all rates above 20% would be reduced by one tenth every two years, with the final reductions back to 20% coming in 1842. Protectionism as a principle was not abandoned and provisions were made for raising the tariff if national interests demanded it.[81]

Although not specifically linked by any negotiated agreement, it became clear that the Force Bill and Compromise Tariff of 1833 were inexorably linked. In his February 25 speech ending the debate on the tariff, Clay captured the spirit of the voices for compromise by condemning Jackson's Proclamation to South Carolina as inflammatory, admitting the same problem with the Force Bill, but indicating its necessity, and praising the Compromise Tariff as the final measure to restore balance, promote the rule of law, and avoid the "sacked cities", "desolated fields", and "smoking ruins" he said the failure to reach a final accord would produce. The House passed the Compromise Tariff, 119–85, and the Force Bill, 149–48. In the Senate, the tariff passed 29–16 and the Force bill 32–1, with many opponents of it walking out rather than voting.[82]

Calhoun rushed to Charleston with the news of the final compromises. The Nullification Convention met again on March 11. It repealed the November Nullification Ordinance and also, "in a purely symbolic gesture", nullified the Force Bill. While the nullifiers claimed victory on the tariff issue, even though they had made concessions, the verdict was very different on nullification. The majority had in the end ruled and this boded ill for the South and its minority's hold on slavery.[83] Rhett summed this up at the convention on March 13. Warning that "A people, owning slaves, are mad, or worse than mad, who do not hold their destinies in their own hands," he continued:

    Every stride of this Government, over your rights, brings it nearer and nearer to your peculiar policy. ... The whole world are in arms against your institutions ... Let Gentlemen not be deceived. It is not the Tariff—not Internal Improvement—nor yet the Force bill, which constitutes the great evil against which we are contending. ... These are but the forms in which the despotic nature of the government is evinced—but it is the despotism which constitutes the evil: and until this Government is made a limited Government ... there is no liberty—no security for the South.[84]

Aftermath

People reflected on the meaning of the nullification crisis and its outcome for the country. On May 1, 1833, Jackson predicted, "the tariff was only a pretext, and disunion and Southern confederacy the real object. The next pretext will be the negro, or slavery question."[85]

The final resolution of the crisis and Jackson's leadership had appeal throughout the North and South. Robert V. Remini, the historian and Jackson biographer, described the opposition that nullification drew from traditionally states' rights Southern states:

    The Alabama legislature, for example, pronounced the doctrine "unsound in theory and dangerous in practice." Georgia said it was "mischievous", "rash and revolutionary". Mississippi lawmakers chided the South Carolinians for acting with "reckless precipitancy".[86]

The historian Forrest McDonald, describing the split over nullification among proponents of states' rights, wrote, "The doctrine of states' rights, as embraced by most Americans, was not concerned exclusively, or even primarily, with state resistance to federal authority."[87] But by the end of the nullification crisis, many Southerners questioned whether Jacksonian Democrats still represented Southern interests. The historian William J. Cooper Jr. notes, "Numerous Southerners had begun to perceive it [the Jacksonian Democratic Party] as a spear aimed at the South rather than a shield defending the South."[88]

In the political vacuum created by this alienation, the Southern wing of the Whig Party was formed. The party was a coalition of interests united by the common thread of opposition to Jackson, and more specifically to his "definition of federal and executive power." The party included former National Republicans with an "urban, commercial, and nationalist outlook", as well as former nullifiers. Emphasizing that "they were more southern than the Democrats," the party grew within the South by going "after the abolition issue with unabashed vigor and glee." With both parties arguing who could best defend Southern institutions, the nuances of the differences between free soil and abolitionism, which became an issue in the late 1840s with the Mexican War and territorial expansion, never became part of the political dialogue. This failure increased the slavery issue's volatility.[88]

Richard Ellis argues that the end of the crisis signified the beginning of a new era. Within the states' rights movement, the traditional desire for "a weak, inactive, and frugal government" was challenged. Ellis writes, "in the years leading up to the Civil War the nullifiers and their proslavery allies used the doctrine of states' rights and state sovereignty in such a way as to try to expand the powers of the federal government so that it could more effectively protect the peculiar institution." By the 1850s, states' rights had become a call for state equality under the Constitution.[89]

Madison reacted to this incipient tendency by writing two paragraphs of "Advice to My Country", found among his papers. It said that the Union "should be cherished and perpetuated. Let the open enemy to it be regarded as a Pandora with her box opened; and the disguised one, as the Serpent creeping with his deadly wiles into paradise." Richard Rush published this "Advice" in 1850, by which time Southern spirit was so high that it was denounced as a forgery.[90]

The first test for the South over slavery began during the final congressional session of 1835. In what became known as the Gag Rule Debates, abolitionists flooded Congress with petitions to end slavery in the District of Columbia, where states' rights was not an issue. The debate was reopened each session as Southerners, led by South Carolinians Henry Pinckney and John Hammond, prevented the petitions from even being officially received by Congress. Led by John Quincy Adams, the slavery debate remained on the national stage until late 1844, when Congress lifted all restrictions on processing the petitions.[91]

Describing the legacy of the crisis, Sean Wilentz writes:

    The battle between Jacksonian democratic nationalists, northern and southern, and nullifier sectionalists would resound through the politics of slavery and antislavery for decades to come. Jackson's victory, ironically, would help accelerate the emergence of southern pro-slavery as a coherent and articulate political force, which would help solidify northern antislavery opinion, inside as well as outside Jackson's party. Those developments would accelerate the emergence of two fundamentally incompatible democracies, one in the slave South, the other in the free North.[9]

For South Carolina, the legacy of the crisis involved both the divisions within the state during the crisis and the apparent isolation of the state as the crisis was resolved. By 1860, when it became the first state to secede, it was more internally united than any other Southern state. Historian Charles Edward Cauthen writes:

    Probably to a greater extent than in any other Southern state South Carolina had been prepared by her leaders over a period of thirty years for the issues of 1860. Indoctrination in the principles of state sovereignty, education in the necessity of maintaining Southern institutions, warnings of the dangers of control of the federal government by a section hostile to its interests—in a word, the education of the masses in the principles and necessity of secession under certain circumstances—had been carried on with a skill and success hardly inferior to the masterly propaganda of the abolitionists themselves. It was this education, this propaganda, by South Carolina leaders which made secession the almost spontaneous movement that it was.[92]
  '''

topics1 = ['Ocean ecosystems', 'US Politics', 'Civil War', 'Tariffs', 'States Rights', 'Federal Authority', 'History', 'US History']

In [130]:
text5 = '''
Kendrick Lamar Duckworth (born June 17, 1987) is an American rapper and songwriter. Regarded as one of the most influential hip-hop artists of his generation, and one of the greatest rappers of all time, he is known for his technical artistry and complex songwriting. He was awarded the 2018 Pulitzer Prize for Music, becoming the first musician outside of the classical and jazz genres to receive the honor.

Lamar began releasing music under the stage name K.Dot while he was attending high school. He signed with Top Dawg Entertainment (TDE) in 2005, where he co-founded the hip hop supergroup Black Hippy. Following the success of his alternative rap debut album Section.80 (2011), Lamar secured a joint contract with Dr. Dre's Aftermath Entertainment and Interscope Records. He rose to prominence with his gangsta rap-influenced second album Good Kid, M.A.A.D City (2012), which became the longest-charting hip hop studio album in the Billboard 200's chart history; Rolling Stone named it the greatest concept album of all time. Lamar's third album, To Pimp a Butterfly (2015), infused hip-hop with historical African-American music genres such as jazz, funk and soul. It became his first of five consecutive number-one albums on the Billboard 200 chart, and was one of the most critically acclaimed albums of the 2010s.

In 2015, Lamar garnered his first number-one song on the Billboard Hot 100, after featuring on the remix of Taylor Swift's "Bad Blood". His critical and commercial success continued with his R&B and pop-leaning fourth album Damn (2017), yielding his second Hot 100 number-one single, "Humble". He curated original songs for the soundtrack album of the 2018 film Black Panther, earning a nomination for the Academy Award for Best Original Song for the US and UK top-ten single "All the Stars" (with SZA). Lamar's 2022 double album Mr. Morale & the Big Steppers concluded his tenure with TDE and Aftermath. His feud with Drake and sixth album GNX (2024) spawned the Hot 100 number-ones "Like That", "Not Like Us" and "Squabble Up".

Lamar has received various accolades throughout his career, including 17 Grammy Awards (the third-most won by a rapper), a Primetime Emmy Award, a Brit Award, four American Music Awards, six Billboard Music Awards, 11 MTV Video Music Awards (including two Video of the Year wins) and 37 BET Hip Hop Awards (the most won by any artist). Time listed him as one of the 100 most influential people in the world in 2016. Two of his concert tours, the Damn Tour (2017–2018) and the Big Steppers Tour (2022–2024), are amongst the highest-grossing rap tours in history. Three of his works were included in Rolling Stone's 2020 revision of the 500 greatest albums of all time. Outside of music, Lamar co-founded the creative company PGLang and ventured into film with his creative partner, Dave Free.
Life and career
Early life

Kendrick Lamar Duckworth was born on June 17, 1987, in Compton, California.[2] He is the first child of former gang hustler Kenneth "Kenny" Duckworth and hairdresser Paula Oliver.[3] Both of his parents are African Americans from the South Side of Chicago.[3] When they were teenagers, they relocated to Compton in 1984, due to his father's affiliation with the Gangster Disciples.[4] Lamar was named after singer-songwriter Eddie Kendricks of the Temptations.[5] He was an only child until the age of seven and was described as a loner by his mother.[3][6] Eventually, his parents had his two younger brothers and younger sister, businesswoman Kayla Sawyer (née Duckworth).[7] His cousins include basketball player Nick Young and rapper Baby Keem.[8][9]

Lamar and his family lived in Section 8 housing, were reliant on welfare and food stamps, and experienced homelessness.[10][11] Although he is not a member of a particular gang, he grew up with close affiliates of the Westside Pirus.[4] Despite suffering hardships, Lamar remembered having "good memories" of his childhood that sparked his interest in hip hop music, such as sneaking into his parents' house parties.[3][12] He was raised secular, although he occasionally attended church services and was taught the Bible by his grandmother.[13] He felt "spiritually unsatisfied" as a child due to the "empty" and "one-sided" nature of the sermons.[14]
Lamar experienced the 1992 Los Angeles riots during his childhood.

After hearing a recording of his voice for the first time, Lamar became interested in rapping.[15] He was introduced to police brutality after experiencing the first day of the 1992 Los Angeles riots.[3] When he was five years old, he witnessed a murder for the first time while sitting outside of his apartment unit, as a teenage drug dealer was killed in a drive-by shooting.[3] "It done [sic] something to me right then and there," Lamar later admitted to NPR Music. "It let me know that this is not only something that I'm looking at, but it's something that maybe I have to get used to."[16] His parents nicknamed him "Man-Man" due to his precocious behavior, although he confessed it "put a stigma on the idea of me reacting as a kid sometimes—I would hurt myself and they would expect me not to cry."[17]

In school, Lamar was a quiet and observant student who excelled academically and had a noticeable stutter.[18] His first grade teacher at Robert E. McNair Elementary School encouraged him to become a writer after she heard him correctly use the word "audacity".[19] As a seventh grade student at Vanguard Learning Center, Lamar was introduced to poetry by his English teacher, Regis Inge.[20] Inge integrated the literary form into his curriculum as a response to the growing racial tensions amongst his students.[20] Through its connection to hip hop, Lamar studied rhymes, metaphors and double entendres, which made him fall in love with songwriting: "You can put all your feelings down on a sheet of paper, and they'd make sense to you. I liked that."[3][20] Instead of completing assignments for other classes, Lamar would scribe lyrics in his notebooks.[20] His initial writing was entirely profane, but it helped him manage his psychological trauma and depression, which he struggled with during his adolescence.[20][21] Inge played a vital role in his intellectual growth, often critiquing his lexicon and suggesting prompts to strengthen his prose.[20]

Lamar later attended Centennial High School.[22] He was enrolled in summer school during the tenth grade, which he dreaded because it forced him to be embroiled in a gang war.[22] Despite his efforts to avoid them, Lamar soon became heavily involved with Compton's hedonistic gang culture, which led to numerous health scares and encounters with the police.[3] He distanced himself from the lifestyle following an intervention staged by his father.[6] When he was 16, Lamar was baptized and converted to Christianity following the death of a friend.[23][24] He graduated from high school in 2005 as a straight-A student.[25][26] He flirted with the idea of studying psychology and astronomy in college, but suspended his academic pursuits to focus on his music career.[3][27]
2003–2008: Career beginnings
Lamar began his career while he attended Centennial High School

During high school, Lamar adopted the stage name K.Dot and began freestyling and battle rapping at school.[28] His performances caught the attention of fellow student Dave Free, who traveled from Inglewood to watch him rap.[28] They quickly formed a friendship over their love of hip hop and the television sitcom Martin.[28] They recorded music together at Free's makeshift garage studio and at his older brother's Hyde Park apartment.[28] Lamar's earliest performances were held at a "super hood" comedy club and behind a tattoo parlor.[28] Free was his hype man during that time, while his older brother was his manager and disc jockey.[28] Lamar recorded five mixtapes throughout the 2000s; his first, Youngest Head Nigga in Charge (Hub City Threat: Minor of the Year), was released on April 15, 2003, through Konkrete Jungle Musik.[29] The mixtapes primarily consisted of freestyles over the production of popular hip hop songs.[29]

In a series of retrospective reviews for Rolling Stone, Mosi Reeves complimented Lamar's "unerring" sense of rhythm and timing found in Hub City Threat: Minor of the Year, but criticized his "clumsy" lyricism and that his flow was "overly beholden to ... Jay-Z and Lil Wayne".[29] Free, who was working as a computer technician, introduced the mixtape to record producer Anthony "Top Dawg" Tiffith while attempting to repair his computer.[30] Tiffith was impressed with Lamar's burgeoning abilities and invited him to partake in an audition process for entry into his newly established independent record label, Top Dawg Entertainment (TDE).[30] During his audition, Lamar freestyled for Tiffith and record executive Terrence "Punch" Henderson for two hours, a strategy that impressed Henderson but bewildered Tiffith.[6][31] He was offered a recording contract by TDE in 2005, joining Jay Rock as the label's first signings.[6] Upon signing, he purchased a minority stake in the label for an undisclosed amount.[31]

Lamar had a brief stint as a security guard when he started working on music with Jay Rock at TDE's in-house recording studio.[32][26] The bond he formed with him, Ab-Soul and Schoolboy Q led to the formation of the hip hop supergroup, Black Hippy.[33] Lamar released his second mixtape, Training Day, on December 30, 2005.[29] Reeves complemented its varied production and "well-executed" concept, which was based on the 2001 film of the same name.[29] In 2006, Lamar signed an artist development deal with Def Jam Recordings and was featured on two singles by the Game. He also heavily contributed to Jay Rock's first two mixtapes, Watts Finest Vol. 1 and Watts Finest Vol. 2: The Nickerson Files.[17][34] Lamar was ultimately let go from Def Jam after an encounter with its president and chief executive officer, Jay-Z; he later described it as "one of those situations where I wasn't ready."[35][36] Lamar and Jay Rock released a collaborative mixtape, title No Sleep 'til NYC, on December 24, 2007.[29] Reeves thought the project was a "fun cypher session, nothing more, nothing less."[29]
2009–2011: Overly Dedicated and Section.80

Lamar's third mixtape C4, released on January 30, 2009, is a tribute project to Lil Wayne's Tha Carter III (2008) and was supported by his co-sign.[37] Reeves felt that the mixtape was a "wrongheaded homage to a year-old, well-worn album."[29] From February to July, he toured with the Game on his LAX Tour as a hype man for Jay Rock.[38][39] Lamar disliked how his stage name diverted attention away from his true identity, and decided to retire it.[40] He opted to use his first and middle names professionally and regards the name change as part of his career growth."[41] For his eponymous debut extended play (2009),[42] Lamar eschewed the creative process of his mixtapes in favor of a project heavily focused on his songwriting over "lovely yet doleful" production.[29] Reeves described the EP as the "first standout project" of his career, praising its melancholic tone.[29] He felt that the project restored his reputation following the sting of criticism he received over C4.[29]
Lamar performing at Sound Academy in 2011, prior to the release of Section.80

After striking a music publishing deal with Warner/Chappell Music,[43] Lamar released his fourth mixtape, Overly Dedicated, on September 14, 2010. It was his first project to be purchased through digital retailers.[44] Reeves described Overly Dedicated as a partial "victory lap" that marked a shift in his songwriting.[29] The mixtape peaked at number 72 on Billboard's Top R&B/Hip-Hop Albums chart.[45] Lamar served as Jay Rock's hype man for a second time during Tech N9ne's Independent Grind Tour, where Overly Dedicated was introduced to Dr. Dre.[46][47] After watching the music video for the song "Ignorance Is Bliss" on YouTube, he reached out to Lamar with hopes of working with him and Snoop Dogg on his unfinished album, Detox.[46][48] He also considered signing him to his record label, Aftermath Entertainment, and was encouraged to by artists such as J. Cole.[49][50]

Lamar entered a brief relationship with Nitty Scott,[51][52] and was featured on XXL's 2011 Freshman Class list.[53] He released his debut studio album, Section.80, on July 2, 2011,[54] which was supported by its lead single "HiiiPower".[55] The album explored conscious and alternative hip hop styles and experimented with "stripped-down" jazz production.[56][57] Ogden Payne of Forbes considers it to be "the genesis to [Lamar] successfully balancing social commentary with mass appeal."[58] Section.80 marked Lamar's first appearance on the Billboard 200 chart, where it peaked at number 113. It sold approximately 5,000 copies in its first week of tracking, with minimal coverage from mainstream media outlets.[59] To promote the album, Lamar performed at small venues and college campuses across the U.S.[60][61] He was dubbed the "New King of the West Coast" by Snoop Dogg, Dr. Dre and the Game during a performance in West Los Angeles.[62][63] Throughout the year, he appeared on the Game's The R.E.D. Album, Tech N9ne's All 6's and 7's, 9th Wonder's The Wonder Years, and Drake's Take Care.[64]
2012–2013: Good Kid, M.A.A.D City
Lamar performing at the Pitchfork Music Festival in 2012

Lamar began planning his second album before Section.80 was released.[65] From February to April 2012, he opened for Drake on his Club Paradise Tour.[66] He began working with J. Cole on a collaborative album around that time.[67][68] On March 8, The Fader reported that Lamar had signed a joint venture recording contract with Aftermath Entertainment and Interscope Records; under the deal, TDE continued to serve as his primary label.[69] His first commercial single, "The Recipe" featuring Dr. Dre, premiered on rhythmic crossover radio on April 2.[70]

Good Kid, M.A.A.D City, Lamar's second album and first under a major record label, was released on October 22, 2012.[71][72] He worked with producers such as Pharrell Williams, Hit-Boy, Scoop DeVille, Jack Splash, and T-Minus to create an atmospheric West Coast hip hop album with heavy gangsta rap influences.[73] Its lead single, "Swimming Pools (Drank)",[74] marked Lamar's first top 20 single on the U.S. Billboard Hot 100.[75] Its other singles, "Backseat Freestyle", "Poetic Justice", and "Bitch, Don't Kill My Vibe", enjoyed moderate commercial success.[76][77] Good Kid, M.A.A.D City was met with widespread critical acclaim, who lauded Lamar's nonlinear songwriting and thematic scope. Greg Kot of the Chicago Tribune applauded him for giving gangsta tropes a "twist, or sometimes upend[ing] them completely" on a record that "brims with comedy, complexity and the many voices in [Lamar's] head."[78] The album debuted at number two on the Billboard 200 with 242,000 copies sold;[79] the highest first-week album sales of the year by a male rapper.[80] Good Kid, M.A.A.D City surpassed The Eminem Show (2002) to become the longest-charting hip hop studio album on the Billboard 200.[81] In October 2022, it became the first hip hop studio album to spend over ten consecutive years on the chart.[82]

From September to October 2012, Lamar headlined the BET Music Matters Tour with Black Hippy and Stalley.[83] He won Lyricist of the Year at the BET Hip Hop Awards,[84] and was featured on ASAP Rocky's single "Fuckin' Problems" alongside Drake and 2 Chainz, which reached the top 10 in the U.S.[85] Lamar embarked on two headlining concert tours in 2013: a national college tour with Steve Aoki and his first international tour.[86][87] He struggled with depression, survivor's guilt and suicidal ideation during promotional events upon learning of the deaths of three close friends.[88] From October to December 2013, Lamar opened for Kanye West on his Yeezus Tour, despite disapproval from his label and management team.[89][90] He was baptized for a second time during the beginning of the tour, and experienced a nervous breakdown near the end.[91][92] Lamar won three awards each during the BET Awards and BET Hip Hop Awards,[93][94] including Best New Artist at the former.[95][96]

Lamar was featured on six songs throughout the year: "YOLO" by the Lonely Island featuring Adam Levine,[97] the remix of "How Many Drinks?" by Miguel,[98] "Collard Greens" by Schoolboy Q,[99] "Control" with Big Sean and Jay Electronica,[100] "Give It 2 U" by Robin Thicke featuring 2 Chainz,[101] and "Love Game" by Eminem.[102] His performance on "Control" was described as a "wake up call" for the hip hop industry and commenced his decade-long feud with Drake.[103] Rolling Stone noted that his verse made the track one of the most important hip hop songs of the last decade.[104] Lamar was named Rapper of the Year by GQ during their annual Men of the Year edition.[105] Following the issue's release, Tiffith pulled him from performing at GQ's accompanying party and accused Steve Marsh's profile on him of containing "racial overtones".[106][107]
2014–2016: To Pimp a Butterfly and Untitled Unmastered

After his opening stint for the Yeezus Tour ended, Lamar started working on his third album.[90] He earned seven nominations at the 56th Annual Grammy Awards (January 2014), including Best New Artist, Best Rap Album, and Album of the Year for Good Kid, M.A.A.D City.[108] He was winless at the ceremony, which several media outlets felt was a snub.[109][110] Macklemore, who won Best New Artist and Best Rap Album, shared a text message that he sent Lamar after the ceremony ended, in which he apologized for winning over him.[111] The incident was the subject of widespread media attention, controversy and Internet memes.[112] During the awards ceremony, Lamar performed a mashup of "M.A.A.D City" and "Radioactive" with rock band Imagine Dragons, which was met with critical acclaim.[113][114]

Lamar opened for Eminem on the Rapture Tour from February to July 2014.[115][116] On August 9, he premiered the short film M.A.A.D, which he starred in, commissioned and produced, during the Sundance Institute's Next Fest.[117] He released "I" as the lead single to his third album, To Pimp a Butterfly, on September 23, which won Best Rap Performance and Best Rap Song at the 57th Annual Grammy Awards.[118][119] His performance of "I" during his appearance as a musical guest on Saturday Night Live was lauded by contemporary critics.[120] Lamar was featured on three songs in 2014: "It's On Again" by Alicia Keys,[121] "Babylon" by SZA,[122] and "Never Catch Me" by Flying Lotus.[123] He won Lyricist of the Year for the second consecutive time at the BET Hip Hop Awards.[124]
Lamar at the Hollywood Palladium before the 57th Annual Grammy Awards (2015), where he won Best Rap Performance and Best Rap Song for "I"

Originally scheduled to arrive at a later date, To Pimp a Butterfly was released on March 15, 2015.[125] The album incorporated various genres synonymous with African American music, such as jazz, funk, and soul.[126] To capture its essence, Lamar recruited producers such as Sounwave, Pharrell Williams, Terrace Martin, and Thundercat.[127] Whitney Alford, Lamar's high school sweetheart, contributed background vocals on select tracks.[128] Other singles from the album were "The Blacker the Berry",[129] "King Kunta",[130] "Alright", and "These Walls"–all of which enjoyed moderate commercial success.[131] Selling 324,000 copies in its first week, To Pimp a Butterfly became Lamar's first number-one album on the Billboard 200 and the UK Albums Chart.[132][133] Billboard commented that "twenty years ago, a conscious rap record wouldn't have penetrated the mainstream in the way [Lamar] did with To Pimp a Butterfly. His sense of timing is impeccable. In the midst of rampant cases of police brutality and racial tension across America, he spews raw, aggressive bard while possible cutting a rug."[134] Pitchfork opined that the album "forced critics to think deeply about music."[135]

Lamar announced his engagement to Alford in April 2015.[136][137] He earned his first number-one single in the U.S. through the remix of singer-songwriter Taylor Swift's "Bad Blood".[138][139] It won Video of the Year and Best Collaboration at the MTV Video Music Awards, while the music video for "Alright" won Best Direction.[140] Lamar later re-recorded his featured appearance on the "Bad Blood" remix in support of Swift's counteraction to her masters dispute.[141][142] He opened the BET Awards with a controversial performance of "Alright" and won Best Male Hip Hop Artist.[143] He also won three awards at the BET Hip Hop Awards.[144] In support of To Pimp a Butterfly, Lamar embarked on the Kunta's Groove Sessions Tour, which ran from October to November 2015 in select intimate venues across the U.S.[145] For his work on the album and other collaborations throughout the year, Lamar earned 11 nominations at the 58th Annual Grammy Awards, the most by a rapper in a single night.[146] He led the winners with five awards: To Pimp a Butterfly was named Best Rap Album, "Alright" won Best Rap Performance and Best Rap Song, "These Walls" won Best Rap/Sung Performance, and "Bad Blood" won Best Music Video.[147]

During the ceremony, Lamar performed a critically acclaimed medley of "The Blacker the Berry", "Alright", and an untitled song.[148] He previously performed untitled songs on The Colbert Report (December 2014) and The Tonight Show Starring Jimmy Fallon (January 2016).[149][150] After receiving a request from basketball player LeBron James to share the untitled works, Lamar released his first compilation album, Untitled Unmastered, on March 4, 2016.[151] It contained eight untitled, dated, unfinished, and entirely self-written tracks that were intended to be included on To Pimp a Butterfly, and continued the album's exploration of jazz, funk, soul, and avant-garde styles.[152] Untitled Unmastered received critical acclaim and debuted atop the Billboard 200 with 178,000 album-equivalent units, becoming Lamar's second consecutive number-one project.[153] Throughout the year, he was featured on four commercially successful songs: Beyoncé's "Freedom",[154] Maroon 5's "Don't Wanna Know",[155][156] the Weeknd's "Sidewalks",[157] and Travis Scott's "Goosebumps".[158]
2017–2019: Damn and Black Panther: The Album
Lamar on the Damn Tour (2017)

On March 1, 2017, during a cover story for T, Lamar confirmed that he was working on his fourth album, Damn.[159] He released the promotional single "The Heart Part 4" on March 23, before releasing the album's lead single "Humble" on March 30.[160][161] The song debuted at number two on the Hot 100 and reached the top spot in its second week of charting. It is Lamar's second single, and first as a lead artist, to top the chart.[162] Damn was released on April 14.[163] It utilized a more mainstream musical palette than To Pimp a Butterfly, exploring R&B and pop elements.[164] Rolling Stone described its sonics as a "brilliant combination of the timeless and the modern, the old school and the next-level."[165] Damn became Lamar's most commercially successful album. It spent four non-consecutive weeks atop the Billboard 200, marking his third consecutive number-one album, and debuted with 603,000 units sold.[166][167] All of the album's 14 songs debuted on the Hot 100, including the top-20 singles "Loyalty" and "Love".[168] Damn was the seventh best-selling album of 2017, according to the International Federation of the Phonographic Industry (IFPI), while "Humble" was the sixth best-selling single of the year.[169] By June 2018, it became the first album by a rapper or solo artist to have every song featured earn a gold certification or higher from the Recording Industry Association of America.[170]

To support Damn, Lamar embarked on his first headlining arena tour, the Damn Tour, from July 2017 to July 2018.[171] It grossed $62.7 million in worldwide revenue, becoming one of the highest-grossing hip hop tours in history.[172] At the 2017 MTV Video Music Awards, Lamar opened the ceremony with a performance of "DNA" and "Humble".[173] He later won five awards, including Best Hip Hop Video, Best Direction, and Video of the Year for "Humble"; the latter win marked the first time an artist won the prize for a video they co-directed.[174][175] Throughout the year, he was featured on the remix to Future's "Mask Off",[176] SZA's "Doves in the Wind",[177][178] and Rich the Kid's "New Freezer".[179] He won Best Male Hip Hop Artist at the BET Awards,[180] while Damn won Favorite Rap/Hip Hop Album at the American Music Awards.[181] A collector's edition of the album, which featured its tracklist in reverse order, was released in December.[182][183]

On January 4, 2018, Lamar announced that he would be curating and executive producing Black Panther: The Album, the soundtrack from the 2018 film of the same name.[184] It was released on February 9 and was supported with three commercially successful singles: "All the Stars",[185] "King's Dead",[186] and "Pray for Me".[187][188] Lamar contributed lead and background vocals to every track on the album, regardless of credit, and produced on select songs.[189][190] Music critics consider Black Panther: The Album to be a milestone achievement, giving praise towards its lyrics and cultural significance.[191][192] It spent two consecutive weeks atop the Billboard 200,[193] and earned the most single-week streams for a soundtrack album in history.[194] Lamar opened the 60th Annual Grammy Awards with a critically acclaimed medley.[195] He won five awards during the ceremony: Damn was named Best Rap Album, "Humble" won Best Rap Performance, Best Rap Song, and Best Music Video and "Loyalty" won Best Rap/Sung Performance.[196] Damn won the Pulitzer Prize for Music on April 16, 2018, marking the first time a musical composition outside of the classical and jazz genres received the honor.[197][198]

From May to June 2018, Lamar co-headlined The Championship Tour with several TDE artists.[199][200] While on tour, he became embroiled in a public dispute with Spotify regarding the streaming service's Hate Content & Hateful Conduct policy.[201][202] Lamar was featured on five songs throughout the year: "Dedication" by Nipsey Hussle,[203][204] "Mona Lisa" by Lil Wayne,[205] "Tints" by Anderson .Paak,[206][207] and "Wow Freestyle" by Jay Rock; he also executive produced the latter's album Redemption.[208][209] At the American Music Awards, Black Panther: The Album won Favorite Rap/Hip-Hop Album.[210] Lamar made his acting debut as a drug addict in the crime drama series Power (2018).[211] After his two concert tours ended, he entered a four-year recording hiatus;[212] although he contributed to Beyoncé's The Lion King: The Gift, Schoolboy Q's Crash Talk, and Sir's Chasing Summer (all 2019).[213][214] As his publishing deal with Warner/Chappell Music was beginning to expire,[43] Lamar signed a long-term worldwide deal with Broadcast Music, Inc.[215] He and Whitney Alford welcomed their first child, Uzi, on July 26, 2019.[216][217]
2020–2023: Mr. Morale & the Big Steppers

On March 5, 2020, Lamar and Dave Free launched the creative entity PGLang, which was described at the time as a multilingual, artist-friendly service company.[218][219] In October, he signed a worldwide administration agreement with Universal Music Publishing Group.[220] Lamar announced through an August 2021 blog post that he was in the process of producing his final album under TDE, confirming rumors that emerged last year that he would be leaving to focus on PGLang.[221][222] The following week, he appeared on Baby Keem's single "Family Ties", which won Best Rap Performance at the 64th Annual Grammy Awards.[223][224] Lamar made additional contributions to Baby Keem's album The Melodic Blue by providing background vocals and appearing on the song "Range Brothers".[225] In November, he held a "theatrical exhibition of his musical eras" during his second headlining performance at Day N Vegas,[226][227] and featured on Terrace Martin's album Drones.[228] He co-headlined the Super Bowl LVI halftime show alongside Dr. Dre, Snoop Dogg, Eminem, 50 Cent, and Mary J. Blige on February 13, 2022, which won the Primetime Emmy Award for Outstanding Variety Special (Live).[229][230]

After releasing the promotional single "The Heart Part 5",[231][232] Lamar's fifth album, Mr. Morale & the Big Steppers, was released on May 13, 2022.[233] He and Alford used the album's cover art to announce the birth of their son, Enoch.[234][235] The double album drew on jazz, R&B, trap, and soul influences;[236][237] Alford served as its primary narrator.[238] It was widely acclaimed by critics, who applauded Lamar's vulnerable songwriting and scope.[239] Every track from the album charted on the Hot 100; its three singles–"N95", "Silent Hill", and "Die Hard"–debuted in the top-10.[240] Selling 295,000 units in its first week, Mr. Morale & the Big Steppers became Lamar's fourth consecutive number-one album on the Billboard 200.[241] It later became the first hip hop album of the year to reach one billion streams on Spotify.[242]

In support of Mr. Morale & the Big Steppers, Lamar embarked on the Big Steppers Tour, which ran from July 2022 to March 2024.[243] The tour grossed $110.9 million in worldwide revenue, becoming the highest-grossing rap tour ever at the time.[244] Lamar wrote, co-directed, and executive produced the short film adaptation of the song "We Cry Together", which was released worldwide in September 2022.[245] An accompanying concert film for the tour, Kendrick Lamar Live: The Big Steppers Tour, was released in November.[246][247] Lamar won Favorite Male Hip Hop Artist at the American Music Awards, and Favorite Hip Hop Album for Mr. Morale & the Big Steppers. He received six awards at the BET Hip Hop Awards, including Album of the Year.[248][249] During the 65th Annual Grammy Awards, Mr. Morale & the Big Steppers was named Best Rap Album, while "The Heart Part 5" won Best Rap Performance and Best Rap Song.[250]

In May 2023, Lamar was featured on the standalone version of Beyoncé's single "America Has a Problem" and appeared on Baby Keem's single "The Hillbillies".[251][252] He won four awards at the BET Hip Hop Awards, and set four records in the process.[253] Lamar was featured in the documentary concert film Renaissance: A Film by Beyoncé and executive produced Baby Keem's short film adaptation of The Melodic Blue.[254][255] He quietly shedded his ties with Aftermath Entertainment and signed a new direct licensing agreement with Interscope.[256]
2024–present: Feud with Drake, GNX and Super Bowl LIX halftime show
Further information: Drake–Kendrick Lamar feud § 2023–2024: Escalation, and GNX (album)

Lamar's conflict with Drake re-escalated in March 2024 with his surprise appearance on Future and Metro Boomin's track "Like That".[257] The song spent three consecutive weeks atop the Billboard Hot 100, becoming Lamar's third number-one single and his first song to debut at the top spot.[258] From April to May, he released the Drake-aimed diss singles "Euphoria",[259] "6:16 in LA",[260] "Meet the Grahams",[261] and "Not Like Us"; all of which were either positively received or acclaimed by critics.[262] The latter installment marked the first rap song to lead the Hot 100 with a limited tracking week.[263] A celebratory one-off concert, titled The Pop Out: Ken & Friends, was held on Juneteenth.[264] On September 8, 2024, it was announced that Lamar would headline the Super Bowl LIX halftime show, marking the first time a rapper has led the performance as a solo act.[265] Three days later, he released an untitled song to his Instagram account.[266] On November 22, he shared a song titled "GNX", exclusively on YouTube, followed with a surprise release of the album of the same name on streaming the same day.[267][268] On December 17, he released a song called "Money Without Me" on YouTube, rumored to be from Section.80 sessions.

Outside of music, Lamar starred in the animated biographical film Piece by Piece (2024).[269] He signed on to produce a comedy feature with Free, Trey Parker and Matt Stone for Paramount Pictures, which is slated to be released on July 4, 2025.[270]
Artistry
Influences
Lamar considers Tupac Shakur to be his biggest influence.

Tupac Shakur is Lamar's biggest influence, having impacted both his professional and personal lives.[271] One of his earliest childhood memories is watching him and Dr. Dre film the second music video for their single "California Love" with his father at the Compton Swap Meet.[12] Lamar has described himself as an "offspring" of Shakur's artistry and sociopolitical views.[272] Although some publications regard him as the Shakur of his generation,[273][274] he strives to maintain his individuality.[275]

Shakur's The Don Killuminati: The 7 Day Theory (1996), The Notorious B.I.G.'s Life After Death (1997), and DMX's It's Dark and Hell is Hot (1998) influenced Lamar's artistic direction: "I don't look at these albums like just music; it sounds like an actual film."[276] He also listened and took influence from Mos Def and Snoop Dogg during his childhood,[277] and said, "I wouldn't be here today if it wasn't for Eazy-E."[278] 50 Cent's mixtape success inspired Lamar to become an independent artist,[6] while his view on being categorized as a conscious rapper, "Yeah, I'm a conscious artist because I have a conscience," gave him a sense of perspective.[279]

Prodigy of Mobb Deep was a key influence on Lamar's earlier mixtapes,[280] while his rapping technique was stemmed from Lil Wayne and his longevity.[281] Eminem and his album The Marshall Mathers LP (2000) introduced him to songwriting elements, such as ad-libs, and impacted his aggressive approach to records such as "Backseat Freestyle".[282][283][284] He took inspiration from N.W.A's tenacity of representing his hometown with "courage, honesty and artistic brilliance."[285] Various R&B and soul artists, including Marvin Gaye,[286] the Isley Brothers,[286] Michael Jackson,[287] Teddy Pendergrass,[288] Sade, and Anita Baker, have influenced Lamar.[289] He performed with Prince, who impacted his vocal register,[290] at Paisley Park to celebrate the release of the latter's 2014 albums Plectrumelectrum and Art Official Age, which GQ described as "five minutes of brilliant insanity."[291] To Pimp a Butterfly was influenced by the works of jazz trumpeter Miles Davis and funk collective Parliament-Funkadelic.[292]
Musical style

The nature of Lamar's musical style has been described as "anti-flamboyant, interior and complex."[293] He is rooted in West Coast hip hop,[294] and has continually reinvented his sound by branching out into other genres.[295] Due to his contributions to its audience growth, through his appeal to mainstream listeners, music critics generally categorize Lamar as a progressive rap artist.[296] He suggests that his music is genreless, explaining in a 2012 interview, "You really can't categorize my music, it's human music."[297] PopDust opined that during the 2010s, a decade that was arguably defined by hip hop, Lamar constantly pushed the boundaries of what the genre could be.[298]

Lamar did not care for music production during the beginning of his career.[299] However, as he placed an emphasis on songwriting and "making material that's universal", he grew more exacting and adventurous with his compositions.[299] He is heavily involved with every aspect of his production process, including the mixing and mastering stages, and is known for working long hours in the recording studio.[300] "You gotta be hands on and know the different sounds and frequencies," Lamar explained to Variety.[299] "What makes people move, what melodies stick with you, taking the higher octaves and the lower octaves and learning how to intertwine that in a certain frequency, how to manipulate sound to your advantage."[299] Lamar chooses to work with a close-knit team of musicians, rather than constantly seek high-profile talent.[46] He has been working with his longtime producer, Sounwave, since his 2009 self-titled EP.[29]

Kendrick Lamar marked a pivotal change in Lamar's artistry. Unlike his earlier mixtapes, which consisted of freestyles over CHR and urban radio singles, the EP incorporated melancholic and "doleful" original production that emphasized his lyrics.[29] Austere jazz production was blended with alternative rap styles on Section.80,[301] with instrumentals drawing from R&B, boom bap, psychedelia, and downtempo.[302] Good Kid, M.A.A.D City abandoned the tastes of contemporary hip hop by exploring a subtle, atmospheric side of West Coast hip hop and gangsta rap.[303][304] To Pimp a Butterfly is an amalgamation of genres synonymous with African-American music, most prominently jazz, funk, and soul.[305][306] It redefined jazz rap by highlighting improvisation and soloing rather than primarily using sampling.[307][308] Minimalist arrangements are incorporated in Damn and Mr. Morale & the Big Steppers.[309][310] Damn appealed to mainstream listeners through its pop and R&B-influenced production,[311] while the scattered and distorted instrumentals of Mr. Morale & the Big Steppers was designed to make listeners feel anxious and uncomfortable.[312]
Voice

Several media outlets consider Lamar to be the greatest and most important rapper of his generation.[313][314] Billboard, Forbes and Vibe named him the second-greatest rapper of all time, behind Jay-Z.[315][316] Described as a "blazing" technical rapper and "relentless searcher" by The New York Times,[293] Lamar's "limber, dexterous" flow switches from derivative to generative metrics,[317][318] while incorporating internal and multisyllabic rhyme schemes.[319] His rhymes are typically manipulated within common time, allowing him to subtly control his metrical phonology and suggest formal ambiguities similar to pop and rock repertoires.[319] Some of his rhyme manipulations feature "flexible" new school styles evoking the 1990s, while others use "rigid" old school elements recalling the 1980s.[319] Lamar frequently uses syncopation in his melodies to create contradictions between his lyrical content and rhythms.[320] With Good Kid, M.A.A.D City, he liberally plays with pronunciation, inflections, and delivery to mirror the album's emotional range.[321]

Lamar possesses a versatile tenor vocal range[322][323] and a raspy, half-shout timbre, where "his throat sounds dry and his mouth sounds wet."[324] André 3000 was the first rapper that introduced him to singing sensibilities in hip hop,[325] and he writes melody-driven songs as practice for his albums.[325] Lamar became comfortable with his vocals over time, to the point where he feels confident enough to create singing-based albums.[326] Pitchfork noticed how his harmonies on To Pimp a Butterfly never made him sound alone throughout his "desolate" performance; comparing his vocal layering to "standing in the middle, unnoticed, of a large quarrelsome crowd."[327]

Praised for his willingness to use his voice as an instrument,[328] Lamar adopts different cadences, tones, modulations, and timbres to suggest conflicting personalities, paint distinct emotions, and communicate stories using characters and personas.[329][330] His falsetto register, which he calls the "ghetto falsetto",[325] has been likened to Curtis Mayfield's.[331] MTV writes that by manipulating his voice, Lamar calls back to a lineage that runs through James Brown's foundational work in the 1960s, 1970s psychedelia, Prince's "sweaty" phantasmagoria in the 1980s, and 1990s gangsta rap.[332] He was ranked the tenth-best solo singer of the 21st century by The Times in 2023.[333]
Songwriting
Further information: List of songs recorded by Kendrick Lamar
Martin Scorsese at the 60th Berlinale in 2010
Quentin Tarantin(o in Paris in 2013
Several publications have compared Lamar's songwriting to Martin Scorsese (left) and Quentin Tarantino's (right) screenplays.

Branded as a "master of storytelling" by The New Yorker,[330] Lamar has been referred to as one of the greatest lyricists in modern hip hop by several publications and his peers.[334][335] Pharrell Williams suggests that what makes his songwriting stand out is because he "knows how to be very disciplined with a subject matter, he knows that stickiness is important, and he knows that it has to be great."[336] American Songwriter notes that for as much as Lamar is a musician, lyricist, and emcee, he is also "a playwright, a novelist, a short story author. He's literary within the art form of music."[337] Lamar's reflective narrative songwriting pulls from a wide range of literary and cinematic techniques, such as hip hop skits and voice-overs, to allow his audience to follow internal and external storylines.[338] His fusion of various film styles and his sonic influence has elevated his works to be some of the most "consistently poignant" in hip hop, and promoted the advancement of the narrative device.[338]

Lamar, who self-identifies as a musician and writer,[159] begins his songwriting process with an assortment of premeditated thoughts that he jotted down over the course of one year.[339] His personal experiences are a common source of inspiration, but he also pulls ideas from meeting new people, traveling, and experiencing different cultures.[339] A devout Christian, he additionally shares his spiritual triumphs and struggles on his songs.[340][341] He is an avid note-taker, and has developed keywords, phrases, and sounds to help him "trigger the exact emotions" he felt when writing the initial demo.[339] Considered to be a "radio-friendly but overtly political rapper" by Pitchfork,[342] and a populist by The Wall Street Journal,[343] Lamar's songwriting regularly infuses political criticism and social commentary concerning African-American culture.[344] Common themes explored include racial inequality, institutional discrimination, and black empowerment.[345] Lamar's critiques has been compared to the State of the Union Address by The Guardian,[346] while Billboard described it as "Shakespearean".[347] HuffPost opined that his work is a "great" piece of journalism because it "speaks from the prerogative of black communities facing oppression and directly attacks the institutions responsible for their pain," an achievement most reporters cannot attain.[348]

Lamar tries to carry a conceptual idea inside of his music, "whether it's a big concept or it's so subtle you can't even tell until you get to 20 listens."[299] Fans and publications have theorized that his albums are related to different forms of mass media.[349] Section.80 is regarded as a short story collection inspired and themed around events that impacted the millennial generation, such as Ronald Reagan's presidency.[350][351] The nonlinear narrative structure of Good Kid, M.A.A.D City is billed as a coming-of-age short film that chronicles Lamar's harsh teenage experiences in his native Compton.[352][353] Its cinematic scope has been compared to the screenplays written by filmmakers Martin Scorsese and Quentin Tarantino.[354][355] To Pimp a Butterfly unfolds as both a poem and blank letter that explores the responsibilities of being a role model and documents life as an African American during Barack Obama's presidency.[356][357] Damn is labeled as an introspective satire that explores the dualities of human nature and morality.[358][359] Mr. Morale & the Big Steppers takes on the form of a theatrical play, with confessional lyrics based on Lamar's experiences in therapy.[360][361]
Reception
Legacy
Lamar performing at the Festival Internacional de Benicàssim in 2016

As one of the most influential musicians of the 2010s decade, Lamar has been deemed a paradigm shift in contemporary hip hop and popular culture.[362][363] His discography became a catalyst in the upsurge of social conscience across multiple generations; challenging the status quo by encouraging listeners to reexamine social institutions.[364] Throughout the Black Lives Matter movement and events following the 2016 U.S. presidential election, his work has been used as protest anthems.[365] According to American studies and media scholar William Hoynes, Lamar's progressive elements places him amongst other African American artists and activists who "worked both inside and outside of the mainstream to advance a counterculture that opposes the racist stereotypes being propagated in white-owned media and culture."[366] He has been credited with reviving jazz rap and the music video as a form of social commentary.[367][368]

Lamar's music has consistently garnered critical acclaim and commercial success—a rare combination in the music industry—as well as support from artists who have paved the way for his advancement, earning him the nickname "King Kendrick".[369][370] His Pulitzer Prize win was considered a sign of the American cultural elite formally recognizing hip hop as a "legitimate artistic medium".[371] Senior artists such as Nas,[372] Bruce Springsteen,[373] Eminem,[374] Dr. Dre,[375] Prince,[376] and Madonna have praised his musicianship.[377] David Bowie's final album, Blackstar (2016), was inspired by To Pimp a Butterfly, and its producer Tony Visconti praised Lamar as a "rulebreaker" in the music industry.[378][379] Pharrell Williams called him "one of the greatest writers of our times" and likened him to Bob Dylan.[380] Lamar has also been cited as a strong influence on the works of various modern artists,[381] including BTS,[382] Dua Lipa,[383] Tyler, the Creator,[384] Roddy Ricch,[385] and Rosalía.[386] Lorde regards him as "the most popular and influential artist in modern music."[387]
Public image
Further information: Drake–Kendrick Lamar feud

Despite becoming a prominent figure in popular culture, publications have noted Lamar's unconventional approach to celebrity culture.[388] He is notoriously reserved, reluctant to publicly discuss his personal life and generally avoids using social media.[389][390] He is also decisive when engaging with mainstream media outlets, although journalists have complimented his "Zen-like" calmness and down to earth personality.[391][392] According to Lamar, he has become "so invested in who I am outside of being famous, sometimes that's all I know. I've always been a person that really didn't dive too headfirst into wanting and needing attention. I mean, we all love attention, but for me, I don't necessarily adore it."[393] His lyrics have been a topic of media scrutiny, leading to both praise and controversy.[16][394][395]

Lamar's public perception has also been influenced by the various rap feuds he has been involved with.[396][397] Although journalists unanimously declared him the winner of his highly publicized conflict with Drake,[398] some felt that his victory was pyrrhic due to the severity of accusations introduced and the spread of online misinformation.[399][400] Following the release of Good Kid, M.A.A.D City, media outlets have described Lamar as the "modern hip hop messiah".[401] Some critics dislike his "grating" political infusions,[402] causing him to be viewed as having a savior complex.[403][404] However, Lamar has declared himself to be the "greatest rapper alive" due to his personal connection to hip hop.[405] "I'm not doing it to have a good song, or one good rap, or a good hook, or a good bridge," he explained to Zane Lowe. "I want to keep doing it every time, period. And to do it every time, you have to challenge yourself and you have to confirm to yourself—not anybody else, confirm to yourself that you're the best, period. [...] That's my drive and that's my hunger, I will always have."[406]
Other ventures
Entrepreneurship

Lamar has been described as an "authentic" businessman who takes "calculated steps to establish his brand from the ground-up" and leaves nothing to chance.[407] He approaches traditional album rollouts with an unorthodox method, using creative Easter eggs and leaving cryptic messages.[408] Before releasing a studio album, Lamar shares a promotional single taken from "The Heart", a timestamp song series designed to "observe the beating pulse behind his music."[409] The vulnerable themes explored on the non-album singles have strengthened his relationship with his "inquisitive" fanbase known as Kenfolk.[409][410] His real estate portfolio includes properties in California and New York.[411][412] In 2011, Lamar crafted an original song with record producer Nosaj Thing to promote Microsoft's Windows Phone in 2011.[413] He starred alongside DJ Calvin Harris and singer Ellie Goulding in a marketing campaign for Bacardi in 2014.[414]

As a minority shareholder of TDE, Lamar was set to serve as the executive producer for the label's film division.[31] He partnered with American Express on advertising campaigns for Art Basel and Small Business Saturday,[415] and is an angel investor of the music creation platform EngineEars.[416] Lamar has also partnered with several fashion designers and outlets. As a brand ambassador, he was involved with designing sneakers for Reebok and Nike.[417][418] He developed working relationships with Grace Wales Bonner and Martine Rose; through their respective eponymous brands, they have dressed him for several public events.[419] For her Autumn/Winter 2023 collection, Twilight Reverie, Lamar worked with Bonner to create the show's soundtrack with Sampha and Duval Timothy.[420][421] Through PGLang, he composed the score and co-designed the stage for Chanel's Spring/Summer 2024 haute couture collection.[422]
Philanthropy and activism

A supporter of the Black Lives Matter movement, Lamar is a vocal advocate for racial equality.[423] In 2012, he commended Frank Ocean for coming out and endorsed Barack Obama's presidential re-election campaign, claiming his opponent Mitt Romney does not have a "good heart".[424][425] Lamar developed a strong friendship with Obama,[426] having worked on a promotional video for Obama's My Brother's Keeper initiative and performed at his Independence Day celebration at the White House.[427][428] He was critical of Donald Trump's first presidency and the U.S. Supreme Court's landmark decision to overturn Roe v. Wade.[429][430]

Lamar has headlined charity concerts benefitting local and international non-profit organizations.[431][432] He donated to the American Red Cross in November 2012 to support victims of Hurricane Sandy.[433] In December 2013, Lamar donated $50,000 to his alma mater, Centennial High School, in support of its music department.[434] He embarked on a small concert tour in 2014, and donated all of the revenue to Habitat for Humanity and his hometown.[435] In July 2017, Lamar purchased a wheelchair-accessible van for a quadriplegic fan.[436][437] He has regularly performed at TDE's annual holiday toy drive at Nickerson Gardens,[438][439] and organizes his own toy drive in Compton.[440] He joined a peace walk in June 2020 to protest against the murders of George Floyd and Breonna Taylor.[441][442] In June 2024, Lamar spearheaded a $200,000 donation to 20 charities and community initiatives based in Los Angeles.[443]
Achievements
Further information: List of awards and nominations received by Kendrick Lamar
Lamar (left) at the 2016 White House Independence Day celebration with President Barack Obama (center) and singer Janelle Monáe (right)

Throughout his career, Lamar has won 17 Grammy Awards (the third-most by a rapper in history),[444][445] a Primetime Emmy Award,[446] four American Music Awards,[447] 37 BET Hip Hop Awards (the most won by any artist),[448] 11 MTV Video Music Awards (including two Video of the Year wins),[449] 6 Billboard Music Awards,[450] and a Brit Award.[451] As a songwriter, he has received nominations for an Academy Award and a Golden Globe Award.[452][453] At the 58th Annual Grammy Awards, Lamar received the most Grammy nominations by a rapper in one night, with 11.[454][146] During the 65th ceremony, he became the first artist from any genre to be nominated for Album of the Year with four consecutive lead studio albums since Billy Joel (1979–1983).[455]

Lamar has appeared in various power listings. In 2015, he was featured on Ebony's Power 100 list that honors leaders within the African American community.[456] Time included him on its annual list of the 100 most influential people in the world in 2016.[457] He has appeared on Forbes' Celebrity 100 ranking (2019),[458] and its 30 Under 30 list twice in the music category (2014 and 2018).[459][460] Lamar was included twice in Billboard's lists of the greatest rappers of all time (2015 and 2023).[461][462] Complex named him the best rapper alive twice (2013 and 2017),[463][464] and included him in their list of the 20 best rappers in their 20s thrice (2013, 2015, and 2016).[465] In May 2015, Lamar was declared a generational icon by the California State Senate for his contributions to music and philanthropy.[466] He was a grand marshal for the Compton Christmas Parade,[467] and was presented with the key to the city of his hometown for representing its evolution.[364] He served as Compton College's surprise commencement speaker on June 7, 2024.[468] Lamar is the fifth man to appear solo on the cover of Harper's Bazaar.[469]

Good Kid, M.A.A.D City, To Pimp a Butterfly, and Damn were featured in Rolling Stone's industry-voted ranking of the 500 greatest albums of all time and the 200 greatest hip hop albums of all time.[470][471] Good Kid, M.A.A.D City was additionally featured in the magazine's list of the 100 best debut albums of all time, and was named the greatest concept album ever.[472][473] It was named the seventh greatest album of all time by Apple Music in 2024.[474] To Pimp a Butterfly was ranked by several publications as one of the greatest albums of the 2010s decade,[475] while "Alright" was deemed the greatest hip hop song of the streaming era by Spotify.[476] As of February 2023, it is the top ranked album on the online encyclopedia Rate Your Music.[477] Damn is the recipient of the 2018 Pulitzer Prize for Music, the first time a musical work outside of the classical and jazz genres was honored.[478][479] Its tour companion, along with Big Steppers Tour (2022–2024), are two of the highest-grossing hip hop tours of all time.[480]
'''

topics5 = ['Gaming', 'African American', 'African American Struggles', 'Machine Learning', 'Computer Graphics', 'Rap', 'Hip Hop', 'Musicians', 'The Game']

## Test: Youtube Transcript

In [14]:
text2 = '''
the year is 2022 and a man by the name of George Anthony devala Santos is running for congress in New York's
thirdd congressional district that's this area of Long Island here close to a
million constituents and a very wealthy area this being where the Great Gatsby was set this area is not unimportant in
fact this race could actually come to define the razor thin margins that decides who will hold the House Majority
until 2024 enter political newcomer George Santos he'd rocked up to the political
scene in Long Island a couple of years prior running as a house Republican in 2020 he'd lost but this area had
historically leaned heavily Democrat and he was up against a political heavyweight at the time a man called Tom
swazi but the year was now 2022 and this time your man Tom swazi
wasn't running meaning no current incumbent and that the seat was fair game now about Santos he's chic Burberry
scarves farago shoes Rayband Shades and his stapled sweater under the Blazer
look Santos looks good his education only the finest he went to Horus man a
60k a year private high school before being scouted on a volleyball scholarship to baroo college even more
prestigious here he was a star volleyball player winning championships against Yale and Harvard before
graduating from New York un University with a business degree in 2013 straight
after college he's air dropped directly onto Wall Street where he starts working for Goldman Sachs and City Group
immediately and just last year Santos was managing a $1.5 billion fund at a
company called Harbor City capital not bad he'd also founded and run the
nonprofit pet charity friends of pets United where he'd go on to single-handedly save over 2,000 dogs he
also has a connection to his constituents rarely seen in politics from the basement of Jackson Heights
Queens to Gatsby land Santos was a true New Yorker he's half black half white
and Jewish I come from a Jewish Family my mother's family Jewish uh in fact his grandparents actually survived the
Holocaust because my grandparents survived the Holocaust his mom was even in the South Tower during
91 so when in 2020 the resume of this man lands on Joe Cairo the Republican
Party Chairman's desk in Nassau County which the third congressional district covered about half of he didn't think
twice Santos was the man for the job and after another long uous campaign effort
the third District's 2022 Congressional race is almost over and on the crisp
Wednesday morning of November the 9th the votes are being counted and projections slowly
materialized and it was clear Santos had won he'd got over 140,000
votes that was over 20,000 more than his opponent yikes who concedes within hours
of the count our boy was in he doesn't make it a few weeks before
the troubles start but first come we're going to
Chernobyl stcraft X is a huge open world FP s MMO set in the bright and bubbly
Chernobyl exclusion Zone explore the map capture outposts for your factions
scavenge for loot or just call up the boys and go in against mutants anomalies
and other players in this massive open world and it's free to play they've also
just dropped a pretty hefty update jump into the Epic Northern story line here
you'll experience the harsh conditions of the northern regions where the current four factions face another
threat 10 new locations have been slapped on the map a ton of new animations a new loot spawning system
this allows you to replenish your supplies in various places around the map and is that parkour a huge update to
the graphics engine new lighting and Shadow systems weather systems and fresh
textures mean things now look a lot better NPCs have also had an upgrade they're now a lot better in combat and
can hide behind covers and jump off objects so guys we did it and finally
have a beer and kick back in that nice new shelter of yours where you can craft ammo and medkits not a fan of the way it
looks well pick up a hammer and get to work because it's customizable all of
this just works haven't played stcraft before or returning after a long time
offline fantastic just punch in this code and get a huge bonus
pack on the 19th of December 2022 the New York Times publishes an article
it's about George Santos and it's interesting see as is natural Santos had
said a lot of things on the campaign Trail however take a closer look and
some things didn't exactly add up first off his education horis man baroo
college and NYU but when asked to confirm whether he had actually been there horis man conducted a search they
scoured through Decades of their records for one George Santos however nothing the same for Baro College what
and the same for NYU 2 I've never met this man in my life George Santos was nowhere to be found the same for City
group and Goldman Sachs They too had never heard of the man and that $1.5 billion fund he managed well he did work
for a company called Harbor City but turns out they never had more than $17 million and they're currently being sued
by the SEC for being an actual Ponzi scheme despite his claims it seems he
actually attended a public Middle School in Queens and never graduated high school also when he said he was working
on Wall Street fresh out of college he was actually working over in the industrial part of queens in a call
center for $12 an hour some details of his time here come
out Sammy please listen to the allegedly from the first day at this schol Center
Santos would brag about the wealthy family he came from back in Brazil the expensive property he' amassed in places
like n and his Belgian and Jewish parents turns out both of his parents were actually
Brazilian and despite his family's lavish Property Holdings they were actually odd welfare at the time and had
recently been getting eviction notices he'd also apparently go around his office showing everyone his lovely
holiday pictures like that amazing trip he'd just been on to Dubai one problem
he wasn't actually in any of the photos and they were just some random pictures he'd found online and printed out he'd
also tell people that he for their wedding dresses and other random expenses free of charge shockingly he
never followed through and after the New York Times sift through various public documents and court filings turns out
there's a few issues pertaining to his campaign finances what's this one of these includes a $700,000 loan he'
claimed he had made to his own campaign but later on would backtrack saying any
personal loans to the campaign were not from personal funds if that money turns out to be from someone else that's a
potential felony not good on the trail he also repeatedly claimed he owns a
mansion in Oyster Bay Cove as well as a mansion in the Hamptons however campaign
records show that Santos was actually living in a rented terrorist house in Queens and that he'd never even lived at
the address he was currently registered to vote at also he claimed on multiple occasions that he was Jewish from a jewi
family turns out no no he's not joke I'm Catholic but I'm also Jewish other
claims of his that his mother escaped Tower 2 during 911 my mom was a 911
Survivor she was in the South Tower um and she made it out and died from Ash related cancer this mom wasn't even in
the country in 2001 also that she was the first female executive at a major
financial institution she was a housekeeper at the time that she was prominent in the Republican party and a
donor spanning two decades she'd never donated to any political party and was
never registered to vote in New York she wasn't even a citizen at the time then there's the lovely wedding
wait with a woman Santos had been openly gay for a long time and had never
disclosed a marriage when signing up to run Well turns out that after looking into marriage records he had one back in
2012 and Santos had left that little detail off of his disclosure forms despite his new wife filing her first
divorce request just one year later and then using their marriage to allegedly help gain citizenship everything was
above board and not dodgy at all in December 2022 he said he had experienced
vandalism and assassination attempt the need for a police escort at their house for safety and a threatening letter he
also claims he had his shoes stolen off his feet in the middle of Fifth Avenue but worst of all absolutely no
volleyball this was very easily verifiable as Yale who had claim to beat in a championship
we went to play against Harvard Yale and we slay them never actually had a men's vity
volleyball team during the 2022 race various Republican organizations also
caught wind of some of this and the man was dubbed George scanos one Republican says his run wasn't a campaign more like
a beach party Santos had also claimed he'd helped develop carbon capture technology with carbon capture
technology that I've helped develop that fundraise for in my career the result
chaos Santos's lawyer comes out instantly gentlemen these defamatory claims are simply untrue our boy Santos
is just being smeared by his enemies at the paper he says nonetheless a multitude of Investigations go ahead
these include the local Long Island District Attorney's office and the New York State Attorney General's office who
was looking at a number of issues Joe Cairo comes out he says these are pretty serious issues and that Santos should
have probably address them that's not what he's doing in fact no one even knows where he is at the moment his
phone calls also seem to be bouncing but after a few days he decides to pick up the phone he calls Cairo and he has an
explanation see the whole thing was all just one big misunderstanding he says
Cairo asks about his apparently blatant lies and Santos has an answer he had
graduated from barook it's just someone there probably just searched under the wrong name I guess
probably he's going to hold a press conference next week and get this all LED out that never happens he does
however open Twitter and get to work but turns out his answers aren't going down
well and calls to resign come flooding in from all directions he tried his best
but Santos wasn't digging himself out of this and so the apology tour begins he
talks to Fox News oan the New York Post and more and finally comes clean listen
Lads I lied I made mistakes humans have faults I'm not a fraud or fake please
give me a second chance he also admits that he embellished his resume and that
he's really sorry about that it's now January 2023 new year new Santos honest
clean cut as Straight Arrow he's ready to get to work and represent his people
however things start out Rocky on the second day of the year Brazilian authorities have an announcement they're
Reviving a decades old case against Santos for fraud they claim that from
2008 to 2011 Santos had actually been living in Brazil with his mother during
which time he'd stolen jewelry from multiple people and forged hundreds of dollars of stolen checks he'd then used
that to pay for designer clothing and then dip out from the country it also turns out that Santos's own lawyer in
Brazil at the time wasn't exactly clean either he'd been charged and sentenced to 18 years in prison for a connection
to a gang execution he apparently started studying law while on house arrest then the same week that case is
revived a formal complaint begins to be filled in Congress demanding a house ethics investigation meaning he's now
under local state federal and intern National investigation as well as a
number of calls to resign this is all before he's even sworn in but forget about all of that because it's the
special day and nothing was going to ruin it in the midst of all the chaos on
the 7th of January 2023 George Santos is sworn into Congress he could now finally
serve his [Music] district protests erupt outside his new
office in Long Island immediately demanding he resigns there's also a poll that shows that more than 70% of
Republicans in his district want him out texts and calls to his cell phone are still going unanswered and he's
apparently in hiding members of his own team don't even know where he is he's also dodging swarms of reporters who
were peppered around the halls of Congress looking for him 24/7 they're even holding stake outs outside of his
office and it's actually stopping him from turning up naturally the media are out for blood
every Outlet is after him the New York Times The Washington Post Mother Jones
and more information is coming out on him constantly January 12th 2023 just 5
days after being sworn in and after lots of deliberation so Cairo has decided not
to believe Santos on barook searching under the wrong name and he's now holding a press conference also calling
him to resign his immediate resignation this comes as a new report also claims that he alongside his campaign Treasurer
had set up a political campaign organization where he had used his political connections to scam tens of
thousands of dollars out of other Republicans running guilty or not one thing was absolutely clear here this man
was not resigning he was serving his people whether they liked it or not will you resign I will not 142 people ask for
me to resign I'll resign I will not resign I will be continuing to hold my office elected by the people and so he
continues to act like like a congressman he puts on the suit and he plays the part well he talks Iran and gas prices
and attends meetings in votes he's having
fun if he could just pretend none of this other stuff was happening then maybe it'll go away but the first few
dominoes had fallen and more were on their way it's around the 12th of January that something else comes out
see there's a drag queen in Brazil and they've just posted a picture of themselves with another man in drag this
man is George Santos so turns out when Santos was supposedly going to a
$660,000 a year Horus man School in New York he was actually a drag queen living
in Brazil and the media runs with it at one point performed drag in Brazil I've
never known him as George Santos he says he's very confident Santos is the person
to the right of your screen despite the multiple pictures and videos of him as a drag queen he denies he was a drag queen
claiming I'm a drag queen is categorically false it is it's not though is it it's seemingly confirmed
here in a now deleted 2011 Wikipedia page on him which he'd seemingly written himself under the name Anthony dealer
why would someone invent you not only that turns out according to this he'd
also appeared on Hannah Montana and the Sweet Life of Zach and Cody back in the early 2000s he'd also apparently started
this Hollywood career after being scouted by the big producer behind Independence Day by Steven Spielberg his
first role being in the movie The Invasion in 2009 starring um Thurman and
Alicia silverston that's not a movie there was a movie called The Invasion in
2007 however that features Nicole Kidman and Daniel Craig Independence Day was
never directed by Steven Spielberg the same week is the wikip IIA
page is discovered a disabled military veteran comes out he says that Santos scammed him out of $3,000 in 2016 back
when he was homeless turns out that Santos had set up a charity called friends of pets United back in 2013 its
mission to rescue raise money for and support the well-being of dogs and other
pets this is where he claimed to save over 2,000 dogs about that what he'd actually do
allegedly is drive out to places like rural Pennsylvania haggle with dog breeders hand them a bad check and then
take off with the goods he'd go on to sell these puppies for hundreds a piece FEA adoption events run by his same
charity he'd also apparently set up fundraising events organize some drinks
get a Barbie going and then stick a $50 price tag on the ticket with the proceeds going towards something like an
animal shelter however when it was time to hand over the money Santos was gone
but in 2016 a homeless disabled military Veteran's dog had gotten cancer he needs
a few thousand for his dog's life saving surgery and the man he's just been recommended to help out George Santos
Santos sets up a GoFundMe you look amazing it makes some waves on Facebook
and eventually raises over $3,000 so Santos sends the veteran to a practice
he recommends in Queens but here the surgeon he'd recommended tells him his dog tumor is inoperable so the veteran
tries to claim the money and go to another vet however his calls in text to Santos and Al getting no response and
the donations page has been removed from the site entirely Santos had seemingly
taken the Money and Run turns out he'd also been going under the name Anthony
zovi he was eventually charged in Pennsylvania in 2017 for taking off with
the unpaid dogs however he claimed that his checkbook had simply gone missing
and with that he's dismissed of all charges also both the IRS and the
Attorney General's office in New York and New Jersey have a look through the records and neither were able to find
any evidence that friends of pets United ever actually existed at all around the
same time all of this comes out one of Santos's old roommates and queens speaks to the Press he claims that when he knew
him he'd gone by the name Anthony dealer and that delusions of grandeur were abundant with Santos apparently often
splashing $100 bills at bars only to end up broke in just a few days there are
also claims he stole money from an aunt stole checks from a roommate stole jewelry from friends and that $500 bbery
scarf of his also stolen Santos maintains that it's his scarf at this
point a wealth of Santos stories were out it was almost too much to keep track of a steady flow each week each more
Preposterous than the last and on the 22nd of January Santos
lands in New York so the Press pick up their cameras and head out it's go time
boys they continue to stake out's office and linger around the halls of Congress
snapping up any chance to ask him a question he gets quite annoyed got you guys Don donuts and coffee for all the
hard work you guys do thank you so much for being here and doing your job thank you but clear the cameras love him he's
also an instant punch line on SNL All In All the month of January has
been a rough one in February he continues to roll with the punches on the first day of the month he's forced
to step down from every US House committee he's on basically meaning he can't play a larger role as congressman
and he apologizes again saying he's deeply sorry the same day the government
officially probes his friends of pet United charity a couple days after that a new audio is leaked to the press in it
Santos says he lied to everyone and I've obviously up and Li to him like I liked
everyone else the same day as that it comes out that in 2021 Santos had gone around the campaign Trail telling
potential donors that he was a producer on the musical Spider-Man turn off the dark this is a show that lost investors
$60 million suffered a ton of legal battles and Technical problems and
resulted in multiple actors seriously injured and two days after that a sexual
harassment complaint is submitted by a former Aid Santos denies the allegation I did not calling it comical comical
right enough is enough Lads this guy's got to go and they filed to expel the
man from Congress February not off to a great start either then the expulsion filing doesn't get
very far but in March the house Ethics Committee stages a vote to investigate
Santos this over the multiple allegations that he had engaged in unlawful activity during his campaign
failed to disclose required information on house forms violated Federal conflict
of interest laws and engaged in sexual misconduct and there's a lot on the man's plate right now the vote to
investigate further is unanimous since things are going so well in April Santos
decides it's time to make an announcement lads I'm running again and his election campaign for 2024 is
officially declared but on the 10th of May Santos gets a knock on the
door turns out it's the feds and he's now under arrest he's slapped with 13
federal charges including seven counts of wire fraud three counts of moneya laundering one count of theft of public
funds and two counts of lying to the House of Representatives see during
covid George Santos had claimed some pandemic unemployment money about $24,000 worth one problem he had a job
and was on a 120k salary come on Santos had also just sponsored a bill to claw
back fraudulent Co handouts meaning he just endorsed a bill targeting himself
then there's his campaign funds turns out he'd used a ton of them to buy that Designer wardrobe of his and pay for
credit card bills a lot of this came from two big donors who he had managed to persuade to max out contributions for
for both campaigns sending $200,000 each and finally Santos is charged for
defrauding donors through his political campaign organizations more on those so
at this point it was fairly established campaign Finance Regulators were a jerk
especially with the help of an experienced campaign treasurer for context a campaign Treasurer gives the
green light on all payments keeps records of all receipts and files all reports statements so they're quite
important and very involved and he'd got himself a veteran a woman named Nancy
Marx and as it turns out this campaign Treasurer has a bit of a reputation
she'd done this for multiple campaigns and other clients and word on the street is she likes to secretly Bill herself
thousands of dollars at random while doing it she'd even been sued for this at one point however she won the suit
anyway in May 2021 Santos goes into business with with her to form a political consulting firm in Florida
called red strategies here Santos apparently uses his new friends in politics to shill Red strategies
fundraising and advertising services to other Republican candidates running in New York all the while secretly not
disclosing his own involvement that involvement being running the company once signed up they charged their
massive fees for apparently very subpar efforts and it's gone and then profit
and turns out a lot of that profit is going directly into Santos's Pockets
Santos would go on to ditch marks apparently telling her the brain cancer was back and that it'd have to go radio
silent for a while he then forms Redstone strategies in 2022 with an old
friend from Harbor City and after its open shop Santos's donors start to receive calls Redstone strategies had
just raised 800k for Santos's campaign and they're now asking for around 700k
more the guy on the other end tells them originally these calls are from a representative of the company but
eventually hello it's apparently just santos himself and that would be a bit illegal Marx has since pleaded guilty to
one felon count of conspiracy to defraud the United States and to falsifying and
inflating Financial numbers with Santos's help she faces 3 to four years in
prison Santos then graduates to devala company here instead of trying to sell
an alleged Ponzi scheme to well-off people like he was doing at Harbor City he has a brilliant idea just introduce
them to other well-off people and take a fee for whatever changes hand and what better time to do this just as he was
entering politics he could be proposing these deals to a whole new class of wealth and power using this company
Santos is allegedly able to negotiate the sale of a massive yacht it hasn't infinity pool a water pool and is worth
around $19 million meaning Santos potentially raked in hundreds of thousands of dollars here and this was
just before the election in 2022 he claims that this is his family firm and handles 80 million in assets anyway
Santos's family comes together to pay off his 500k bond for the charges and on the condition he surrenders his passport
he's free to go he and his lawyer go on to shoot down each and every single charge it's a Witch Hunt Santos says
it's a Witch Hunt it makes no sense he pleads not guilty and still refuses to
resign following all of these charges another vote to expel Santos kicks off
however this fails to pass and is instead handed over to the house Ethics Committee to investigate at this point
Republican Congressman are getting very agitated what do you have
to like I said if I could if I could understand you over my colleague
screaming here the reality is better
here's the thing though he's not going anywhere because Republicans need him to maintain their razor thin House Majority
which they hold by just four seats ooh that's kind of small so Santos is safe for now so in June it's time to kick
back Santos attends a congressional baseball game where he's mbed by fans he can literally win the presidential race
right now it's now August and an aid of Santos has just been charged with identity theft he allegedly impersonated
the current Speaker of the house Kevin McCarthy's Chief of Staff while fundraising nonetheless the Speaker of
The house's support for Santos remains strong it's around this point that Santos also misses his financial filings
deadline which he' already received a 90-day extension for October
10th and it's the feds again they're back for 10 more charges this time it's
conspiracy false statements falsification of Records aggravated identity theft credit card fraud and
then some more wire fraud and with the second tranch of charges another scheme of his comes to light see back when
Santos was running he'd had an idea this one was simple note down the identity
and financial information of his campaign donors which they'd already given when contributing then repeatedly
charge their credit cards without them knowing profit these would go into his campaign other campaigns and again
directly in to his own pocket in another scheme Santos and his campaign Treasurer apparently filed reports with
overinflated donations on them this could then be used to acquire financial support from the Republican party which
it was more profit again he pleads not guilty to every charge and the thought
of resigning doesn't even cross his mind this along with the speaker of the house and Santos Ally Kevin McCarthy being
given the boot on the 3rd of October and Santos is now considerably less I'm in danger and it's getting to him
Santos starts to crack become unstable is unacceptable they should not be allow
here he things are not looking good so Santos brainstorms he needs to look good
wi back the court of public opinion Santos has to go that's it a baby and
Santos starts walking the halls of congress with a random baby in his hand ex do not report this baby whose was it
where did it come from from no one knows but when asked he says this you said not
yet as in a reference to I'm not a par I'm not there yet the 1st of November
2023 and it's time for another vote to a Santos the house needs a super majority
vote of 2/3 to do it so it could really go either way here and Santos braces is
for the worst however the vote goes down and turns out he' survived easily in
fact 179 yes yes to 239 no looks like our boy's
bulletproof he made it he
won but wait on the 16th of November the house Ethics Committee is back they'd
spent the last few months scouring through records and conducting interviews and now their investigation
was over and their findings handed in 170,000 pages of documents witness
testimonies and Final cial statements and the verdict was in something's a bit
dodgy here the revelations are numerous he'd spent $1,000 on hotels $442,000 on
flights this includes flights to random places like Kansas and Memphis more than 10 trips to the high-end Manhasset
shopping center $300 on an internet viral Steakhouse and countless expenditures at an Italian restaurants
and queens more about that restaurant see Santos likes this place a lot so
much so that he'd spent tens of thousands of dollars here he'd also claimed to be friends with the family
running it the owner's brother even contributed to his campaign it would also turn out that the same man was
caught in 2019 steering a 63t yacht from the Bahamas towards Florida in an
attempt to smuggle 14 undocumented Chinese migrants and a Bahamian man into the us more than $200,000 worth of
currency was also found stuffed into the bedroom walls all above board something
of Interest also to note a ton of expenditures here were exactly
$199.99 here's the thing if he'd spent just two pennies more he would have had to keep some form of documentation like
a receipt the restaurant is also apparently donating thousands of dollars to his campaign so in essence he's
giving the restaurant money via undisclosed food he's buying through his campaign and that same money is going
back to the campaign he'd end up using this 100 $99 trick dozens of times in
fact he ends up with more of these expenses than any other candidate in the entire United States including at places
that would be very difficult to even spend that amount like a car park where it was impossible to spend this exact
amount of money Santos also used campaign funds on lip fillers Botox rent
holidays makeup weight loss drugs Casino trips around a quar of a million dollars
of cash withdrawals even only fans also his designer clothes compliments of the
campaign too of course this includes the ferrao loice at around $700 the Cartier
tank watch at around $10,000 the bbery Quilted jacket around $400 Louis Vuitton
ties around $200 and his rayb bands also around $200 to illustrate just how
Brazen these violations are another us politician apparently had to go at his staff for ordering some fruit to their
hotel door using campaign funds then there's Santos who was practically living in an her store all in all he'd
apparently spent just around a third of his campaign Budget on Advertising this is apparently far less than normal
naturally he denies a lot of this is it true that you have an only fans page and you can peel a banana with your
feet I don't have one and it's you know I'll I'll I'll indulge you this I just discover what only fans was about 3
weeks ago when it was bought up in a discussion in office and I was I was oblivious to the whole
concept the filings themselves also incredibly dodgy they included multiple
revisions unaccounted contributions unitemized payments totaling hundreds of
thousands of dollars and hundreds of those $199 payments to Anonymous who was
then removed in later filings all of this compounded with two of his campaign staff now pleading guilty for various
crimes and yeah not great so Santos comes out with an announcement Lads
probably not running in 2024 sorry then on the 18th of November 2023
he looks like this towards the end of the month it's
clear another expulsion vote was looming and this time it could be a lot closer
so on the 29th of November Santos gives it his all in a house speech while he
makes his case people fly this balloon outside the capital on the 30th Santos
says he'll be holding a press conference outside the capital the next day and this time he actually does hold one
having made his case inside and outside the capital all Santos could now do was
pray December the 2nd 2023 and Congress members are pulling up to the office for
work today's agenda Santos now again to get this man out the house would have to
vote Yes to the tune of a 2/3 super majority so this could actually go either way even with the recent
Revelations but regardless the votes start coming in and eventually the
results are in Santos is out he's booted out his office
loses his seat in the house and banished from the capital becoming the sixth house Congressman in history to be
expelled and the first in over two decades he makes a hasty Escape
in a politically divided America Democrats and Republicans managed to come together for one thing their utter
disdain of this man but within a couple of days of expulsion a new account appears on Cameo it's Santos and he's
now singing Taylor Swift and roasting people online for money when youed in and he's popular in
fact his $75 asking price is quickly kicked up to $400 a piece it doesn't
take long for him to start earning figures and eventually more money than his congressional salary in February
2024 a special election is held to replace Santos at the time a massive
blizzard hits Long Island with areas getting up to 8 in of snow but regardless tens of thousands show up to
vote and on the 13th of February Tom swazi wins back his seat and this short
bizarre chapter of us politics meets its end or does it
yes it does [Music]
'''

topics2 = ['US politics', 'Contemporary events', 'Scandals', 'George Santos', 'Ocean', 'Donald Trump', 'Corruption in US Government']

In [98]:
text6 = '''
this is a 1oz tin of caviar on average
this tiny tin cost
$120 and this is where it comes from
everyone always ask this one question
why is it so expensive and is it even
worth it so we're traveling all the way
to the largest caviar Farm in America to
uncover the truth and once we're done
we'll taste test the cheapest and all
the way up to one of the rarest and most
expensive cavars in the world but first
let's make some caviar we're with our
main guy here at SAR Nikolai one of the
largest caviar farms in America I'm Ali
lury and I'm the president and CEO of
Zar Nicolai caviar Farm we're going to
see our native breed of white sturgeon
that we raised in the nation's only Eco
certified farm and hopefully make some
award-winning cavier today with you I
love it I also stole his jacket right
cold how are we starting our day today
so it's bright and early it's about 7:00
a.m. we're going to do some ultrasounds
and identify the caviar ready females
and we're going to do the process of
selection once that process is over
we're going to go into the caviar making
area and begin the actual row extraction
caviar making process all right a big
ahead of us and lots to tackle so it's
time to mentally prepare myself all
right you ready right now yeah 7:00 in
the morning it's good the water is just
as cold as outside oh great 40° perfect
maybe I will keep your jacket after all
so I got all suited up to get into the
sturgeon tanks but then ol you let me
know one tiny important detail you know
really in this tank alone we probably
have a million dollars in caviar fish so
really please Joshua don't fck this up I
will do my P that said I got in the tank
anyway and immediately felt a little
overwhelmed by the cold but my main
focus was tread carefully because
there's a lot of expensive sturgeon
swimming around I'm stepping I got a
sturgeon literally right in front of me
I mean the fish are literally like
rubbing up against you they SWAT your
crotch every so often I kind of like it
so what we're going to be doing with the
ultrasound is we're going to get a good
look inside the internal organs of the
fish so we're looking for the ovaries
and we're looking for ripe eggs
themselves it's the most fish friendly
it's the least stress on them and it's
the most humane way we can investigate
that obviously in order to check we need
to catch one so I give it my best shot
I'm about to try holding one see if you
can grab it yourself all right she's
gone
all right we're going to give that
another shot oh perfect a gigantic one
good one for Josh to handle oh I did it
yeah let's go wait wait wait wait no no
I just got you here I just got you here
all right well I'll let the pros do this
so we're kind of looking for that nice
roton belly they do have a sound you can
kind of feel it too oh my god wow it's
so noticeable this fish most likely is
rape so we'll kind of ultrasound hurt so
we're going to go through the belly wow
this is huge yep so that's a ripe female
you can kind of see the fine dots right
beneath that skin layer this is great
news we got caviar guys fish that size
so 120 lb we're going to buy 12 lb of
caviar for that fish wow now that we've
selected some Fales we're going to move
them into a separate tank then we'll do
a final taste test to determine that
those are the fish we want to harvest
this is like the earliest tasting of any
tasting I think we've ever done just to
describe the flavor there's no salt
present whatsoever you really get this
really fresh Oceanic flavor it's not
fishy at all the pearls are literally
still warm I mean this is the freshest
consumable caviar in human existence
it's amazing honestly incredible what
you just got to try is like Day Zero the
process of today is to kind of see the
evolution of flavor right so here you
have raw sturgeon row and that 2% flavor
you got today will grow into that full
flavor profile I'm excited to see the
next round you feeling warmer I actually
do feel warm that's my jacket so Joshua
here we are at our Hatchery all of our
genetics all the future species of white
sturgeon that we produce are all done
right here so there's no impact on the
wild population of fish this is where
the magic happens wow they're so little
you kind of seen now how small the egg
was you had in your hand these fish
respa in April so about 5 months of
growth and then from this size to about
year two that's the fastest growing at
year three we'll sex them and then the
females will go to one tank mes go to
another from there little caviar love
these little guys but they're a handful
as far as constant attention they're the
babies okay ultrasound and final
selection completed now we're moving on
to processing but first one of our
missions today is to learn about what
makes caviar so expensive let's see if
Oli can shed some light just wanted to
thank you for uh giving me my jacket
back and not telling me it was wet so
Oli what goes into making caviar why is
it the price that it is you're on one of
the oldest farms in the world kind of
where the technology to farm sturgeon
came to life and we try to do it as
sustainable as possible we antibiotic
free or steroid free or fish are eating
what fish would eat in the wild a lot of
it is done by hand it's a 10-year-old
fish so that's a labor of love you're
going to be doing some expensive work
today I love it you're going to see if
you're R rain Supreme before we process
the female fish what about the protein
what about the meat of the fish guys
well they use the whole thing so here we
are at the end part of our Smokehouse
we're in NOA tail operation we try to
limit waste as much much as we can this
is our award-winning Applewood smoke a
sturgeon four ingredients sturgeon salt
apple wood and a little bit of brown
sugar low and slow 8 to 12 hours it kind
of tastes like the turkey of the sea you
want to join me yes please unbelievable
this might be the perfect intro to fish
for anybody who may not be a fan of it
what I love about it is it's incredibly
mild it's not fishy but you still have
that Oceanic Umami quality to it it's
steak like it's got really intense smoke
flavor a little bit of sweetness from
the brown sugar it's seasoned
wonderfully it's sort of like like a
nice in between of like a nice smoked
ham and like a jerky and like a
swordfish Allin one really tasty one day
my kids will grow up and know that I've
been giving them smoked sturgeon and it
wasn't smoked ham and turkey but we'll
cross that bridge when they're older or
maybe you never have to now I was time
to get my game face on cuz I'm entering
the inner sanctum where around $250,000
of caviar is processed daily they're
trusting me to help them do this so uh
let's not this up our guide through this
with millions of dollars writing on his
shoulders every week CH and yang he's
kind of a caviar badass every day they
process about 40 sturgeon averaging
about 120 kilos total of caviar 9 lb 9
lb of caviar from that one fish okay I
saw the pros do it I think I can handle
this it's just uh you know dropping an
ounce this would cost hundreds of
dollars and there's a lot more than an
ounce I'll tell you that so what's the
best method of transfer put my hand
right here slide it at the bottom make
sure nothing connected still drive it
this way and this way it'll all come out
in one piece yeah there's something
stuck back here is it okay to pull yeah
just give it a little tug I didn't want
to damage it it's kind of easy there wow
there same thing again unbelievable
everybody's standing around me in case I
drop it they're like please don't get
Dro at this stage I was surprised to
hear that this is now my caviar I'm
responsible for it this is my caviar
there are many other like it but this
one's mine I then stepped into the
caviar room so there's five steps that
have to happen in this room that are
crucial to making this right messing up
any of these steps could totally ruin
what would be hundreds of thousands of
dollars of cavar so we start with step
one screening the caviar step two
rinsing step three drying step four
salting and we got to get it down to the
exact gr and step five the final dry
before we pack the caviar into tins and
have our finished product the moment
we've all been waiting for I finally get
to make my own caviar so essentially
first we're taking the row we're running
them through these fine mesh screens
extracting every last egg as much as
possible without damaging them you have
to be very delicate very careful in this
process I'm assuming I've never done
this before I don't want to destroy
thousands of dollars with the caviar I'm
going to do my best here we go so you
want to press down on it just enough
where the eggs fall right out I'm so
nervous to destroy the caviar you're
pressing harder than I thought you'd
have to I feel like when we think of
caviar we think oh this is going to turn
into a pace but really it's very
resilient I'm kind of shocked I guess it
says a lot about the quality too right
they have all these little flaps in
between them too you got to flip them
over a little bit and then open it up oh
wow okay I'm understanding now I'm
understanding now this is kind of fun I
like it
this is analisia our main egg washer
she'll just kind of rinse around the
bowl to get all the eggs in there and
then as you can see all the impurities
come up and then she'll give it a quick
pour being very careful as to not lose
all the eggs in the bowl kind of like
when you're rinsing rice in a rice
cooker it is right really expensive rice
what she putting in with the droplets so
we do four Rines in total the first two
Rines we put in our mixture to soften up
the water first and after that we just
do Straight water so this would be the
second wash with the mixture wow so
delicate it's just an uous process even
all the way down to rinsing with water
is so detailed you want to do the last
two I'll do that so just pour it in and
kind of mix them up with the water
gently pour it in around the bowl yeah
and make sure to stir up the bottom my
years of doing the right thing and
washing my rice is finally paying off
you see you could probably do it too if
you wash your rice you can take that
bowl and then we'll go over to the third
step which is over here there are very
few people in the world who get to taste
caviar through every part of the life
cycle and I'm excited to say that I'm
one of them we tried it raw straight
from the live sturgeon and now we try
unsalted but rinsed so how does that
change the flavor wow so fresh there's a
little bit of umami there it's got
almost like an egg yolk like creamy pop
in your mouth very fresh water flavor
and texture there's so much flavor
naturally sitting there it's waiting to
be brought out by the salt it's really
nice and remember after we finish up
here we'll be tasting the caviar among
various other levels of caviar Rarity to
see if expensive caviar is really worth
it so my glass has been fogging up after
finally realized the trick is I just
hold my glasses up and then use my
eyelashes as windshield wipers oh my God
I can see I'm about to enter one of the
last stages which is salting the caviar
extremely important it has to be done
perfectly cuz this is going to be going
out to three Michelin star restaurants
for our cavar we do the malis salt which
is lower in salt content we do about
like a 3 and 1/2% salt to our eggs it's
not that it's lightly salted though in
my opinion it's just that it's salted
properly bad caviar is going to be super
salty and you're not going to taste much
else but salt this is Eon he's been
salting caviar for 10 years so he knows
the
stuff A Little Love Taps we want the
weight before and after the salt you're
going to add until it says
13.5% so you got your salt you got your
eggs just got to pour it in my hands are
like frozen to give it a good stir yeah
it's so soft it's soft but yeah they're
really firm you could be kind of rough
with him make sure your hand is all the
way to the bottom like this yeah and
then from time to time bring the bottom
to the top to make sure that every
single eggs get some salt in it you can
fill every individual egg they're very
firm extremely resilient it also feels
kind of like I'm a kid playing with like
a bunch of really tiny Orbeez how are we
feeling so far looking okay it looks
good at me yeah before this sits for a
bit we're going to give this one more
try let's see how you do is this my
verification moment
oh thank you so buttery so rich very
mild the salt level is literally perfect
I mean perfect thank you I tried I also
had an electronic scale telling me
exactly what to do this is actually a
really good fish that's like a reserve
level so good Reserve quality cavr Now
we move on to the tinning what you see
here is the last step right before they
get tined this is Jose what he's doing
is removing any impurities from the eggs
look he spotted that boom impuri is the
size of a grain of sand Jose is catching
and tweezing out by hand don't ask me
why this is expensive this is the
freshest caviar you can get before they
get 10 Jose isn't right if I try a
little bit
oh it's incredible every single step of
the process the caviar is transformed
just this bite alone immediate burst of
flavor the Umami the salt level is
absolutely perfect it's buttery and that
just fresh Oceanic flavor I can eat this
right now Jose can start off and show
you how to pack with 1,700 G and then
you can pack your own 500 G after that
one you see what he's doing he's gently
pressing it down to get rid of all the
air pockets on the bottom and then he'll
pack it on top again my God he Stacks it
so high see how masterful Jose is that
spatula is just an extension of his arm
and by eye he already pretty much knows
what 1700
is 23 it's unbelievable I'm 28 so
verying long time but as I watched that
beautiful tin walk away it was my turn
to try and mimic 23 years of experience
in a single moment it's also not easy
with everybody standing here watching
you like everybody don't be afraid to
kind of press down on it and it kind of
give it like a little smear to cover all
the little her Pockets yeah I need my
guy here how am I doing am I doing okay
he's bullshitting me he's bullshitting
so now at this point you're just giving
a little Mound yeah just a little bit
more that looks about 500 G so just kind
of clean up the edges just a little bit
holy sh how does he do this man but just
when I thought I was in the
clear oh no now I have to start over
Jose look at
this at least I got the practice now
right the tin becomes so slippery and
he's moving so fast I feel like this
one's 10 times messier than the last one
give it like just another little scoop
of caviar and then that should be about
500 g yep yep that should be good right
about there so let's see how close you
are to 500 540 540 okay okay so after
packing it we're just going to take
these we're going to take it to our
sterilizer and then we sterilize for
about like 30 seconds just kind of stand
here and look awkward so
yeah and then we'll just take it out and
then this goes where take it back over
there so these are little particulars
instead of just putting it in I like to
kind of go at an angle and then close it
in like this press it down just a little
if you can see like the bubble where
like some juices come out then that's
good enough we've made it the final step
before I cure my very own caviar I had
no idea how much effort concentration
would be required to get here day in day
out there's tons of people that are
behind this operation but more
specifically it takes years just to get
to the point of even extracting the
caviar let alone all the other things
you need so as i s my label I finally
understood that truly fine caviar is
more than just a price tag I I just made
my own caviar I feel like I'm walking
into tasting this caviar with a
completely different Viewpoint of it and
a whole new Found Love of it you did 12
years in a day you did a great job you
were Relentless our shoes got wet our
socks got wet my jacket got wet but by
the end of it I think we gained an
appreciation for each other's craft that
was incredible thank you so much for
doing this for us and showing us
literally your whole process you ever
decide for a different career path there
may be a future here as our Nikolai team
opens the door to you Joshua okay it's
good to know that there's always a
fallback now let's taste some caviar now
for the big question we've seen the
whole process of caviar but there are
more varieties at many different price
points than you could possibly imagine
that's why I brought in three taste
tessers to try four levels of caviar we
have Martins the World's Strongest Man
from 2019 is he a local college student
and Chef Christian is caviar worth it
and does it get better the more you
spend let's find out the most holistic
way to enjoy caviar just purely as it is
is to do what is called a bump you put a
little dollop on your wrist you suck it
off your wrist and you eat it like that
that said we instead decided to do it on
a chip with a little bit of creme fresh
I think it's the easiest way to
introduce anyone to caviar as long as
the chip is not too salted and not over
fried then you end up with really just a
crunchy vehicle to then taste the caviar
and the creme fresh helps mellow out the
flavor of the Chip And you end up with a
really nice caviar tasting something
that everyone can enjoy have you ever
had caviar before I have okay and your
thoughts on caviar I like
it I love caviar it's nice decent I
don't eat it often I don't go to a
restaurant and like yo I'm about to drop
a bag if it's there and it's offered I'm
taking a bump so we have our first timer
with caviar which I'm pumped for you to
try this is level one we've mixed and
matched these they could be high they
could be low give it a taste let me know
what you think oh I'm excited for
this we're thinking creamy I was
expecting it to be like a little
grittier but that one was like a little
more gel smooth like a lot of people
would assume fish eggs are fishy this
wasn't offensive like it don't like it
love it hate it a little underwhelming
for my taste I want it to pop I want it
to pop I don't mind it it kind of
reminds me of like blue cheese
interesting interesting not as Briny as
I'd like or I'm used to but it's good
yeah it's good it's okay it's okay yeah
okay since you've never had cavier you
may not know where to place your guess
on this but how much do you think a 1 oz
tin is and give me a guesstimate on the
price for that one I want to say that's
$55 $60 like
$50 because it's not that great I I
think it's like $1,000 $11,000 wait
let's let's let's start over $7 million
just to give an idea for that size the
average price is 120 unreal really okay
what do you think this one is at would
you say it's at 120 lower I think this
one's over I just have this Prejudice
thinking that the more expensive
something is the less interesting it's
going to be this is actually the
cheapest
Cav this is a $72 o tin it is the petan
classic duranki I would argue this is
the simplest low EST chillest form of
caviar nice would you say that this is
worth it of my money maybe if someone
bought it for me h no I think that's
worth it for a first time buyer I
wouldn't break the bank okay let's move
on to the next round we're on to level
two give it a try let me know what you
think oh that crunch I heard from here
oh that's good now that is good that one
tastes a little more salty a little more
Briny it's a little bit more on the
creamy side I like flavor on the
brininess it pops better is it better
than the last one yes I like that one a
little bit better better than the last
one and by how much this was a standard
for that first one right there I don't
know how much more I gave it so
incremental but noticeable yes yeah I
think I like the last one better I could
get more Umami from it give me your best
guess on the price for 1 oce of this if
the flavor is salty or more strong I
would assume it's more expensive maybe
90 let's go 65 120 strong guest this is
the SAR Nikolai Reserve gold standard
caviar you'll see it in a lot of
restaurants three Mission St restaurants
and it comes in at $140
would you say that $140 ounz tin is
worth it it is not worth it I like the
taste of that one better so that one
would be more worth it to me the price
is still it's up there up there for
someone who enjoys it isn't just looking
for a bang for their butt if you want
good caviar and you're willing to pay a
solid amount this is I would say the
gold standard let's move on to the next
level we're on to level three give
yourself a
taste oh oh yeah that's what I was
looking for that one tastes more intense
such an interesting flavor to describe
besides like salty I feel like I'm
experiencing the sea right now Oceanic
almost Oceanic that's the word one word
that comes to mind is sharp like in a
crisp way like champagne good level on
brininess a hint of bitterness just a
hint I like it it's a little gritty like
Sandy it's like a solid mix between like
a gel and then yeah kind of like sand
it's got a bit of Pop to it but it also
has got a creaminess to it I like that
one that's my favorite so far do you
think this one is better or worse than
the last one I feel like they're kind of
even just as good but not better it's
very close to the first one the first
one was was very interesting it had
almost like a Herby taste to it this one
is just like pure classic caviar taste
what would you say you think this one is
priced at could be higher could be lower
H I'm going to guess 150 based on the
last one I would say that's like a 90 to
100 range I don't think it's one of the
more expensive ones I would give it 90
and this is the petran SAR Imperial
hybrid Beluga caviar and it comes at
$315 per ounce oh my goodness jeez it's
expensive because it's considered the
cream of the crop also makes the biggest
eggs did you feel like those eggs were
bigger it did feel bigger bigger is not
always better so now that you know the
price is it worth it flavor wise I think
the second one I tried had more depth I
enjoyed the second one more ooh that one
is worth it if I had money ready to
spend on delicious food that's the one I
would have bought compared to the last
one they kind of taste the same to me so
I would say not as worth it huge
statement you just saved so many people
so much money all right we're on to
level
four oh oh it's more creamy it doesn't
have that that pop that the previous one
had but it's way more fishy it almost
has like a pure taste lean towards more
ikura than caviar that one tastes more
fishy to me like an ooni like Flavor
almost Briny Oceanic I like that it
doesn't linger too long like it's just a
perfect amount of funk it will smooth
the pearls didn't really necessarily pop
more so melt I like that would you say
this one was better than the last no
it's different I can't say it's better
but I enjoyed it more I think the
previous one's my favorite cuz I love
that pop I'm craving that when I think
of caviar so I'd say this is my second
favorite give me your guess on what this
one is price T then the saltiness makes
me think it's more expensive I'll just
say like 300 I'd be surprised if it was
more than 200 because of the intense
flavor I'm going to put this one at 200
this is almma albino acetra this is the
rarest caviar on Earth because only one
albino sturgeon is born out of every
10,000 sturgeon and it comes in at a
$445 per ounce 10 wow jeez out of all
the ones I've tasted I priced that the
most and I I think shows I didn't know
it was me that much but it definitely
tasted more expensive than the rest of
them this is extra special because it
comes from Maris caviar's affiliate Farm
in Florida it's the only legal American
producer of purebred Beluga caviar the
most prized style of caviar in the world
and now for the last round of
deliberation is this caviar worth it as
a regular consumer sorry it's not as a
minimum wage worker no but if it was
offered to me like this yes anything is
worth it if it's free yes we then asked
each taste tester to rank each level
from best to worst the second one is my
top third first fourth three four 1 2 in
order 4 2 3 1 so in a way the white
albino was still up there for you I
enjoy The Taste the most but the second
one being almost consistent and for that
price point I would go for that over and
over and it's the only one I think worth
its price tag wow that's a banger I love
it big thank you to everyone who tastes
so what's the result is it worth it to
buy caviar and spending more money does
it get you better caviar I think the
simple answer is there's no real
correlation it comes down to the
individual's preference so yeah maybe
spending a little more gets you a
slightly better caviar but even like
Martin said the cheaper one he loved but
you guys rent is so much more important
pay rent first caviar second maybe
actually groceries that fill you maybe
if it's your first time buying caviar or
even if you're a seasoned veteran start
small and work your way up love you so
much subscribe bye
[Music]
[Music]
y
[Music]
'''
topics6 = ['Culinary Arts', 'Cooking', 'Farming']

## Trash

In [16]:
text3 = '''
metal gear rising sons of delivery and
digiorno is my favorite political and
economic simulator well intended to
teach the player the basics of price
slashing and how to best divide the
country as quickly as possible the
answer of course is manually in this
visually engaging stock market index we
play as jack the wood chipper a man
probably on a quest to renew the job
market one life at a time so if you've
seen part one of my philosophically
charged video essay on the deep and
interesting metal gear lore which i
totally understand then welcome back and
if you're viewing this series for the
first time well that's technically okay
you're just gonna be more confused than
an american at a salad bar
[Music]
last time on metal gear raisin guns of
the priests
but as for now our protagonist ryden
with biden is forced to argue with a
decapitated head about memes that's not
made up i know the rest is made up but
that's genuine apple bottom jeans ryden
what boots with the fur
fresh off his adventures to the orange
dimension and the insides of american
police it's up to our hero to avenge
obama's death through the slaughter of
the unmelonated learning the true
meaning of based through many trials
tribulations and his guest seat at the
joe rogan podcast finally finding
himself at the entrance to amazon
warehouses with manageable stab wounds
Amazon Headquarters
damn it i need a token black person what
the [ __ ] he and peel this bloodloss has
made me delirious tell me a story there
once was a silence
holy [ __ ] ryden heads into the hotel
lobby hosted by mr mosby
which the game sends the atf to stop me
from freeing the second amendment i am
forced to take matters into my own hands
this was back when turret sections were
the pinnacle of game design excuse me
fellas i'm just heading to this normal
sized elevator ryden i'm playing another
gotcha game okay please destroy all
records of my credit card debt before
the police come welcome one in holocaust
to a classic stealth game named metal
gear solid 2. we're currently undergoing
a secret mission to destroy scandalous
snake ryden we we have to prevent joe
biden's america you have to stop
you're too late corn pop i have already
vocoded your voice to gangster's
paradise we use our batman vision to
carefully locate the bank records and
also the rocket launcher this puzzle is
a great way to simulate the real office
experience if you work for blizzard so
after expunging the records of overwatch
2 ryden runs through [ __ ] black
friday this makes vietnam look like
clown college we even have a richard
dixon
as you can see the situation has
escalated
what is up gamers maxor here and today
we're going to be entering the um the
what the [ __ ]
so it turns out that bezos is a real big
fan of japanese culture like a really
big fan of japanese culture to the
extent that he filled this building with
ninjas don't worry they still use
rockets the decor is so goddamn oriental
it's killing dolphins ryden traverses
the hidden leaf village with the dk crew
and enters the first elevator built and
designed with elephants in mind or just
your mom on which we fight a gauntlet of
enemies for several minutes as i reenact
my trips to the humane society you will
never catch me nor will you find the
weapon how tall is this [ __ ] building
this is half of the level so we get off
of mr bone's wild ride which allows us
to booze and cruise all the way to the
moon apparently what is that music up
here
oh galio geogel brand i can't wait to
see what the game has in store for me
now after all those long combat arena
what so i want everyone to meet my cool
and original boss design named mistral
this is essentially what the patriots
were doing the entire time ai dennis
prager was merely a distraction these
[ __ ] were synthesizing v tubers so
now we're forced to fight their robotic
fleshlight and her huge testicles this
battle serves as a great moment to see
just how far you've come as a player
both in ability and tolerance for the
best game mechanic don't worry though
i'm still very bad
thank god that shit's over now we can
move on
god damn it
really bold move making me fight monsoon
again after 20 minutes and yeah that
includes the pants attack [ __ ]
skeletron prime looking ass i guess you
can say a good meme never dies jack god
please take me out of this hell i've had
enough memes today look it's shadow and
he's a nice [ __ ]
pretend you didn't see anything today
well well well if it ain't defense
unhand these brains jeff they're
children and kids are cruel jack they
just lose touch with it at the ripe old
age of 12. wait what war crime this
can't eat the drywall that microwave and
mice is wrong they say what the [ __ ] not
to work kids love video games that's why
i got them all set up spinning the wheel
on my favorite gotcha games
how else is a man supposed to make his
impact you refrigerated a preschool for
[ __ ] jpegs
you think i made the market for gone you
jack like every casino is just a big old
conspiracy
[ __ ] gambling's just a part of who
we are i just market it to sexy children
besides demand for my products is about
to skyrocket like the good old days
after 9 11. wait a minute 911 is bad
speak for yourself let's debate this on
the roof
looks like you're a gotcha player at
heart is that because of my brain damage
no it's because you go after children
and there ain't enough room in this
preschool for the two of us
like i said kids are cruel jack and i
love my face
Red Sun
welcome to the bastard featuring his tax
shield this battle is comprised of two
distinct phases each one easier than the
last and for some reason a helicopter go
go gadget concealed carry in his first
phase we simulate beating to death a
cheeky goblin using money to compensate
for his lack of human empathy and by the
second phase it's quite obvious why he
does it this means it's up to us to cut
each shield individually so we can play
the game and yes you explode when you
miss not only this but his health is the
length of a cambodian life in 1977
meaning once you get past that the fight
is basically over in fact it's quite
easy to just not get past the shield's
bever and straight up kill him the
fundular entertainment way featuring
sheldon cooper from the big bong
sorority
but if you do this you're basically
annihilating all enjoyment from orbit i
personally recommend standing there and
just waiting for jeff to up present so
enjoy that second phase while you can
because this music is a gift from god
[Music]
this is as bad [ __ ] as it looks
[Music]
can you tell that's my favorite song
i upgrade ryden's item as we board a
helicopter being driven by a uh
doctor how did you get an aircraft oh
credit card of course by the way ryden
do you remember that bank you sent to
the past tense it was like carving a
pumpkin well i think you missed one
i can't go to prison right and why
because you play genshin no ryden i'm a
nazi
well time to kill myself what the [ __ ]
are you talking about there's so much
hearts of iron to play i'm gonna not see
you in a second
not today i guess so doctor tries to
kill ryden anyways and he responds by
dividing the public
the hard way
私は絶望しています
so ryden hits his head so badly that he
lands in a a different game the volume
is a little bit
of you know it's just slightly uh
it's kind of loud the graphics are a
little different ryden heads to the gas
station and uses the zandatsu to cut
through the environment dynamically yeah
that uh that seems like how it works
this isn't even metal gear rising
anymore i'm just playing roblox games
ryden survives the impact of a sedan
when i was a child and gets utterly
destroyed in denver again just like me
on a saturday night and is forced to
play a previous level but backwards
thank you level designer
oh [ __ ] a free bike
don't worry it's a rental
there are rumors that the nords are
attempting to capture the whole of
soul's time
anyone looking for work should consider
the fighters guild i hear they're always
looking for new recruits you ran over a
child nine miles back what are you going
to do good day
oh so now you decide to stop i'm running
late oh don't worry ryden the clown
college is closer than you think yeah
it's right in front of me
kill yourself your ass will always be
cringe and my ass will always be thick
497 north parker tribe oh good
you're getting more based by the day but
i live with my mother is this her house
The Only Thing I Know for Real
in this duel between two equally matched
swordsmen of various speeds it's up to
us to defeat the weakest brazilian male
this guy's out here making the
rainforest look like a [ __ ] joke and
the best part is there's no [ __ ] in
this fight no minions no shields no
monsoon telling me about [ __ ] among
us it's just you the sunset and the soil
this is the best boss ever holy [ __ ]
ryden disarms wham slam damn sam which
causes him to lose his mcfucking marbles
and this somehow makes the fight harder
i mean he straight up punches you like
he's in goddamn yakuza hey ryden look at
this cool bug i found
alright and no need to flip out
dudes got thunder thighs bigger than the
power grid sam will test every single
neuron in your corpus callosum with
unrelenting and dangerous attacks strong
enough to burn a hole in my adderall
supply it's not a drug problem it's a
drug solution and that is just phase one
god help you when he picks up that sword
again
[Music]
i challenge you to a debate what the
[ __ ] no dies of cringe so ryden goes to
inspect his kill like a cat only to
realize that he has no cybernetic
enhancement this man dunked on ryden
harder than kobe after he threw an ava
unit and he did it by juicing ryden
steals his clifford the big red sword
with a five-star gotcha rarity and
thirty-three percent crit rate why did
you have a motorcycle why are you
sprinting there why don't we just relax
Gums of The Parrots
and turn on the radio would you like him
oh
yes
yes that's good
damn dog give me some of that ryden
mcfucking breaks into nasa so he can go
somewhere quickly you see the current
president george bush has made a horse
race across america that way he can
assemble all of the votes and activate
the guns of the patriot act but to
understand what that means we have to
head to a previous game it's metal gear
solid 4 again and the clinton foundation
controls the entire planet the only
thing standing in their way is the now
unsolid snake because he's old david i
want you to meet someone very important
to me his name is joe biden
i have kept him alive using the miracle
of child sacrifice
and i'm gonna choose death over life
wow that sure made everything easy to
understand oh my god it's my dad oh [ __ ]
up i've returned with a puppy
yeah anyways do you know where daddy can
buy some cigarettes last time you got
those you disappeared for 10 years i
know so have you ever heard of nft
trading no what is that well you start
by buying a monkey can you feed the
monkey nah but you can sell it for a
rocket or some [ __ ] that's great honey
i'll be right back now with our
transportation acquired we blast off
into [ __ ] space our goal is to reach
the utopia of pakistan and stop the evil
empire of america from initiating yet
another military incursion called
operation tecumseh finally i have
الشرق الأوسط
tracked down the great satan to his
source now this cliff will make me a
martyr
[ __ ] jacques already goes to the base so
as to convince the boot boys the error
of their ways and appear on indian news
networks to justify nuclear conflict
bryden have you heard of the hit game
among us you need to vent right and i
know it'll be hard to be science you
might notice something funny about this
level it's uh it's three rooms the
developers [ __ ] gave up so so do i
ryden plays the real game until he finds
the cast of icarly
our whole universe was in a hot dense
state when nearly 14 million years ago
ryden discovers his dog after being made
into a family guy episode no i wanted to
feed you that chocolate before a
gigantic mcfucking crab evolves from the
goddamn soil like a thousand year old
cicada
what the [ __ ]
well if it isn't sassy jack bro are you
high let me check
yes high on american spirit and there's
nothing more american
and there's nothing more american than
shooting a man in this walmart of a
world what is walmart it's heaven ryden
check the internet lately write it hop
on twitter you need to see this [ __ ] i
hate this website it's all [ __ ] weep
[ __ ] right and it doesn't make any sense
they're being distracted without a
nonsense these baboons don't even know
they're at war with pakistan this one's
calling me a redditor and that's just
the beginning e-girls gotcha memes all
just petty distractions so real men can
get down to business
and as you know american imperialism is
absolutely justified because we had a
black president once before i [ __ ]
killed him that's a nice argument
senator why don't you back it up with a
source my sources that i made it the
[ __ ] up imagine a world ride and free of
cancel culture where no one can call me
out for my outlandish claims a world
where i can say the n-word
what
do who's in the gym
i've got my own to debate online
in this battle against microsoft excel
we are faced with a boss too large to be
fun and are relegated to enduring his
onslaught from the inside of the forever
box this battle is a classic case of
wait for him to do anything just like
congress and like congress when he does
do something it usually hurts you
fortunately this music is excellent and
fits one-to-one with all i want for
christmas
[Music]
i'm not joking although for some reason
biden can parry several tons of crush
force maybe my sister could have done
that against my stepdad riding suplex is
the giant lobster for the second time
today then he yanks off his arm to have
a
sword fight with it every other game is
now worse because they don't have this
Proud to be an American
check it out ryden i just got vaccinated
oh god he's hot time to hit the gym
i don't have much i don't know jack
maybe your face could do some work
hope you can afford healthcare ryden
because the taxpayer can't
it's funny because thousands of people
die don't [ __ ] with this senator
[Applause]
land of opportunity american dream it's
all [ __ ] the success of a nation is
determined by materialistic and
socio-economic factors read a [ __ ]
book
the truth then us politicians aren't so
trustworthy we'll steal make [ __ ] up
even lie to our voters that's crazy i
know it's very hard to believe but
that's just what the usa is all about
jack not hiding our theft and murder but
embracing it whether it's votes whether
it's money especially money [ __ ] all
these lipstick youtubers and chicken
[ __ ] redditers [ __ ] this 24 7 internet
spew of discord and feedtuber [ __ ]
i'm gonna build a new future jack one
where we won't have to hide our stealing
from anyone and for that i want you by
my side to build a world truly free from
the consequences of our actions just
like the founding fathers intended
what do you say ryden why not try to say
it
just once
i was wrong about you senator you're not
cringe
you're just [ __ ] racist
[ __ ]
we're making the mother of all omelets
here jack can't fret over every egg i'm
sorry senator but this is my metal gear
rising revengeance
[Music]
you need to work on your fitness what
fitness dick in your mouth
my blood is red white and blue jack i
wonder what color yours is
wait
ryden brian griffin you can't give up
yet think about every mentally deranged
[ __ ] you fought up until this point
you've always had the strength to
overcome whether it was violence in
movies or sex on tv i was always lucky
there was a family guy even if your iq
is the room temperature of alaska you
are the best chia pet ever made sam put
a [ __ ] speaker in his sword that's
right i finally uploaded my
consciousness into my gaming setup ryden
the reason you're cringe is not because
you ain't skilled but because you ain't
ballin on that stank-ass windows 95
pewter even in death he's on crack just
this once i'll lend you my setup so you
can know what it's like to be a real
gamer you're my little dog champ
i've always said my sword was a tool of
justice not a weapon but a way to bring
back those good old-fashioned values
upon which we used to rely but this
isn't my sword
it's a [ __ ] mistake hey what's up
Family Guy Funny Moments
guys maxor here welcome to the final
bond you know he can be a this guy's a
little uh you know he's pretty hard
we're here after extensive cut scenes to
finally teach america that violence is
never a solution it's a question and the
answer is yes start praying boy
dwayne the rock johnson's immense
strength and talent for volcanic
eruptions comes from his ability to
switch between a culture vulture and
obama at will channeling the power of
both races to whoop your ass back to
roblox but among his many racial talents
our buff buzzard is fast has a great
singing voice and possesses forklift
certification when you die it will be in
accordance with osha regulations but
after spending enough time in the oven
our athletic aesthetic disarms ryden and
reports to his chiropractor for an
adjustment this guy beats up more
biracial people than a police officer
armstrong even has the power to receive
the light of allah which bestows upon
him miraculous healing wisdom needless
to say this fight is fantastic combining
and expanding on every skill we've
acquired on our long and arduous journey
for a final battle that truly tests our
metal this isn't even the first time
ryden has stabbed a politician to death
but if i get my way it won't be the
[ __ ] last
[Music]
you know i'm starting to have a change
of heart yeah i can tell that's why
you're the best jack you always wore my
heart on your sleeve
if i hear another goddamn pun if you
kill me jack you'll finally be free
free to follow your heart okay that's it
buddy
and in other news a united states
senator tragically passed away this
morning after a botched open heart
surgery honey can we please stop
watching obituaries you'll be on there
next if you don't shut the [ __ ] up
oh [ __ ] my blackberry good news jonathan
we've got a new job opening for you ah
that's great doc yeah anyways it's for a
ninja cyborg ninja cyborg doesn't ryden
do that oh yes ryden uh-huh well um the
police might be after him for that
vehicular manslaughter
manslaughter doctor i did that [ __ ] on
Conclusion
you're purpose
[Music]
[Music]
i feel like
you
'''
topics3 = ['Metal Gear Rising', 'Nature', 'Sports', 'Conservation', 'Elephants', 'Games', 'ChatGPT']

## Short text

In [205]:
text4 = '''
Elephants are the largest living land animals. Three living species are currently recognised: the African bush elephant (Loxodonta africana), the African forest elephant (L. cyclotis), and the Asian elephant (Elephas maximus). They are the only surviving members of the family Elephantidae and the order Proboscidea; extinct relatives include mammoths and mastodons. Distinctive features of elephants include a long proboscis called a trunk, tusks, large ear flaps, pillar-like legs, and tough but sensitive grey skin. The trunk is prehensile, bringing food and water to the mouth and grasping objects. Tusks, which are derived from the incisor teeth, serve both as weapons and as tools for moving objects and digging. The large ear flaps assist in maintaining a constant body temperature as well as in communication. African elephants have larger ears and concave backs, whereas Asian elephants have smaller ears and convex or level backs.
'''

topics4 = ['Nature', 'Sports', 'Conservation', 'Elephants']

# Add a paraphraser into the pipeline to make it classify better?

In [145]:
model_path = "sshleifer/distilbart-cnn-12-6"
summarizer = pipeline("summarization", model=model_path, tokenizer=model_path, truncation=True, device=-1)
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=-1)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Device set to use cpu
Device set to use cpu


In [138]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
chunks = split_text(text5, max_length=1024)
chunk_summaries = [summarizer(chunk, max_length=300, min_length=100, truncation=True)[0]['summary_text'] for chunk in chunks]
# print(chunk_summaries)
final_summary = " ".join(chunk_summaries)
print(final_summary)

# tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

print(check_long_text_facebook(final_summary, topics5))

Token indices sequence length is longer than the specified maximum sequence length for this model (12483 > 1024). Running this sequence through the model will result in indexing errors
Your max_length is set to 300, but your input_length is only 197. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=98)


 Kendrick Lamar is one of the most influential hip-hop artists of his generation . He was awarded the 2018 Pulitzer Prize for Music . He rose to prominence with his gangsta rap-influenced second album Good Kid, M.A.D City (2012) Rolling Stone named it the greatest concept album of all time . His third album, To Pimp a Butterfly (2015), infused hip hop with historical African-American music genres such as jazz, funk and soul . His fourth album, Damn (2017), yielded his second Hot 100 number-one single, "Humble"  When he was five years old, Lamar witnessed a murder for the first time while sitting outside of his apartment unit, as a teenage drug dealer was killed in a drive-by shooting . His parents nicknamed him "Man-Man" due to his precocious behavior . Lamar was introduced to poetry by his English teacher, Regis Inge, who integrated the literary form into his curriculum as a response to the growing racial tensions amongst his students . In high school, Lamar adopted the stage name K.D

## Me when I don't realize there's a thing exactly for this

In [187]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "cross-encoder/stsb-TinyBERT-L-4"
cross_encoder = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def compute_cross_encoder_similarity(sentence, topics):
    scores = {}
    for topic in topics:
        inputs = tokenizer(sentence, topic, return_tensors="pt", truncation=True, padding=True)
        outputs = cross_encoder(**inputs)
        scores[topic] = outputs.logits[0].item()
    return scores

def check_long_text_cross_encoder(text, topics, threshold=0.25, k=3):
    split_list = split_text(text, max_length=100)
    results = []
    for chunk in split_list:
        # Check the similarities
        similarities = compute_cross_encoder_similarity(chunk, topics)
        # print(similarities)
        results.append(list(similarities.values()))

    scores = np.array(results)
    # top_k_scores = np.sort(scores, axis=0)[-min(k, scores.shape[0]):]    
    zipped = zip(np.max(scores, axis=0), topics)
    sorted_zipped = sorted(zipped, key=lambda x: x[0], reverse=True)
    arr, labels = zip(*sorted_zipped)
    # print("Final arr:", np_arr)
    softmax_scores = np.exp(arr) / np.sum(np.exp(arr))
    print("Labels:", labels)
    print("Scores:", softmax_scores)

    if max(softmax_scores) >= threshold:
      return True
    return False


C:\Users\Linguini\Documents\Personal Projects\Smart-Website-Blocker\venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Linguini\.cache\huggingface\hub\models--cross-encoder--stsb-TinyBERT-L-4. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


# Running the tests

In [240]:
import time

print('############################')
print('############################')
print('           TEST 1           ')
print('############################')
print('############################')
start = time.time()
print(check_long_text_facebook(text1, topics1))
end = time.time()
print(end - start)
print('############################')
print('############################')
print('           TEST 2           ')
print('############################')
print('############################')

start = time.time()
print(check_long_text_facebook(text2, topics2))
end = time.time()
print(end - start)
print('############################')
print('############################')
print('           TEST 3           ')
print('############################')
print('############################')

start = time.time()
print(check_long_text_facebook(text3, topics3))
end = time.time()
print(end - start)
print('############################')
print('############################')
print('           TEST 4           ')
print('############################')
print('############################')

start = time.time()
print(check_long_text_facebook(text4, topics4))
end = time.time()
print(end - start)
print('############################')
print('############################')
print('           TEST 5           ')
print('############################')
print('############################')
start = time.time()
print(check_long_text_facebook(text5, topics5))
end = time.time()
print(end - start)


############################
############################
           TEST 1           
############################
############################
Labels: ('Tariffs', 'States Rights', 'Federal Authority', 'History', 'US Politics', 'US History', 'Civil War', 'Ocean ecosystems')
Scores: (0.9993424614270529, 0.9971094926198324, 0.9919058084487915, 0.9830295840899149, 0.9751088420550028, 0.9122531215349833, 0.773429294427236, 0.5604464610417684)
True
3.117347240447998
############################
############################
           TEST 2           
############################
############################
Labels: ('Scandals', 'George Santos', 'US politics', 'Corruption in US Government', 'Ocean', 'Contemporary events', 'Donald Trump')
Scores: (0.9995777805646261, 0.9936360319455465, 0.977692981561025, 0.577962855497996, 0.3742210076500972, 0.2464640587568283, 0.24024980763594309)
True
1.386577844619751
############################
############################
           TEST 3          

In [214]:
import time

print('############################')
print('############################')
print('           TEST 1           ')
print('############################')
print('############################')
start = time.time()
print(check_long_text(text1, topics1))
end = time.time()
print(end - start)
print('############################')
print('############################')
print('           TEST 2           ')
print('############################')
print('############################')

start = time.time()
print(check_long_text(text2, topics2))
end = time.time()
print(end - start)
print('############################')
print('############################')
print('           TEST 3           ')
print('############################')
print('############################')

start = time.time()
print(check_long_text(text3, topics3))
end = time.time()
print(end - start)
print('############################')
print('############################')
print('           TEST 4           ')
print('############################')
print('############################')

start = time.time()
print(check_long_text(text4, topics4))
end = time.time()
print(end - start)
print('############################')
print('############################')
print('           TEST 5           ')
print('############################')
print('############################')

start = time.time()
print(check_long_text(text5, topics5))
end = time.time()
print(end - start)

Token indices sequence length is longer than the specified maximum sequence length for this model (11635 > 512). Running this sequence through the model will result in indexing errors


############################
############################
           TEST 1           
############################
############################


RuntimeError: expand(torch.LongTensor{[1, 70, 1]}, size=[1, 3]): the number of sizes provided (2) must be greater or equal to the number of dimensions in the tensor (3)

In [226]:
import time

print('############################')
print('############################')
print('           TEST 1           ')
print('############################')
print('############################')
start = time.time()
print(classify_with_distilbert(text1, topics1))
end = time.time()
print(end - start)
print('############################')
print('############################')
print('           TEST 2           ')
print('############################')
print('############################')

start = time.time()
print(classify_with_distilbert(text2, topics2))
end = time.time()
print(end - start)
print('############################')
print('############################')
print('           TEST 3           ')
print('############################')
print('############################')

start = time.time()
print(classify_with_distilbert(text3, topics3))
end = time.time()
print(end - start)
print('############################')
print('############################')
print('           TEST 4           ')
print('############################')
print('############################')

start = time.time()
print(classify_with_distilbert(text4, topics4))
end = time.time()
print(end - start)
print('############################')
print('############################')
print('           TEST 5           ')
print('############################')
print('############################')

start = time.time()
print(classify_with_distilbert(text5, topics5))
end = time.time()
print(end - start)

############################
############################
           TEST 1           
############################
############################
Labels: ('US Politics', 'Federal Authority', 'US History', 'Civil War', 'Ocean ecosystems', 'Tariffs', 'States Rights', 'History')
Scores: (0.86614454, 0.80749637, 0.6616821, 0.5520938, 0.31151813, 0.09838751, 0.03849583, 0.033257104)
True
13.341171741485596
############################
############################
           TEST 2           
############################
############################
Labels: ('Donald Trump', 'George Santos', 'Contemporary events', 'US politics', 'Corruption in US Government', 'Scandals', 'Ocean')
Scores: (0.73575836, 0.6899114, 0.6531034, 0.6298566, 0.62587935, 0.59525293, 0.57401055)
True
6.6108362674713135
############################
############################
           TEST 3           
############################
############################
Labels: ('ChatGPT', 'Sports', 'Nature', 'Games', 'Metal Gear

In [234]:
from transformers import pipeline

# Load the zero-shot classification pipeline with the model
classifier = pipeline(
    "zero-shot-classification",
    model="typeform/distilbert-base-uncased-mnli"
)

# Define input text and possible class names
text = "Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."
labels = ["mobile", "website", "billing", "account access"]

# Perform zero-shot classification
result = classifier(text, labels)

# Print the results
print("Labels:", result["labels"])
print("Scores:", result["scores"])

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Device set to use cuda:0


Labels: ['mobile', 'billing', 'account access', 'website']
Scores: [0.6334249973297119, 0.1339106410741806, 0.12124130874872208, 0.11142311245203018]
